<a href="https://colab.research.google.com/github/Rob174/Astronomy/blob/Astronomy/Modele000002.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:

!pip install -U transformers torch torchvision tensorboardX tf-nightly grpcio==1.24.3
##Python / Colab
from google.colab import files
from google.colab import drive
import os
from IPython.display import Image as imgIPython
from IPython.display import clear_output,display
import IPython
## Tensorflow keras
try:
  !pip install -q tf-nightly
except Exception:
  pass
import tensorflow as tf
from tensorflow.keras import backend as K

from tensorflow.python import debug as tf_debug
from tensorflow.python.client import device_lib

from tensorflow.keras.models import Sequential

from tensorflow.keras.layers import Layer
from tensorflow.keras.layers import Input
from tensorflow.keras.layers import Dense,Conv2D,Convolution2D,Activation,Conv2DTranspose
from tensorflow.keras.layers import MaxPooling2D,AveragePooling2D
from tensorflow.keras.layers import Dropout,Reshape,BatchNormalization
from tensorflow.keras.layers import concatenate,Concatenate,Subtract,Multiply,Average,Add
from tensorflow.keras.layers import UpSampling2D, Reshape,Flatten
from tensorflow.keras.layers import Lambda

from tensorflow.keras.optimizers import SGD, Adam
from tensorflow.keras import models
from tensorflow.keras.models import Model, Sequential
import tensorflow.keras.losses
## Math libraries
import numpy as np
import scipy
import matplotlib.gridspec as gridspec
import matplotlib.pyplot as plt
##Images
from PIL import Image
import cv2
## Graph
from graphviz import render
from graphviz import Digraph,Graph
drive.mount('/content/drive')
%cd '/content/drive/My Drive/TIPE'
#Dataset
from __future__ import absolute_import, division, print_function, unicode_literals
import random
import pathlib
import shutil
import time
#Debugage
from IPython.display import clear_output
#Hyperparameters tuning
# try:
!pip install -U keras-tuner #De https://github.com/keras-team/keras-tuner
# except Exception:
#     pass
from kerastuner import BayesianOptimization
from tensorboard.plugins.hparams import api as hp
#Explainability
try:
    !pip install tf-explain -q
except:
    pass
# !pip uninstall tensorboard
from tf_explain.callbacks.grad_cam import GradCAMCallback
%load_ext tensorboard

Requirement already up-to-date: transformers in /usr/local/lib/python3.6/dist-packages (2.5.1)
Requirement already up-to-date: torch in /usr/local/lib/python3.6/dist-packages (1.4.0)
Requirement already up-to-date: torchvision in /usr/local/lib/python3.6/dist-packages (0.5.0)
Requirement already up-to-date: tensorboardX in /usr/local/lib/python3.6/dist-packages (2.0)
Requirement already up-to-date: tf-nightly in /usr/local/lib/python3.6/dist-packages (2.2.0.dev20200306)
Requirement already up-to-date: grpcio==1.24.3 in /usr/local/lib/python3.6/dist-packages (1.24.3)
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/My Drive/TIPE
Requirement already up-to-date: keras-tuner in /usr/local/lib/python3.6/dist-packages (1.0.1)


#Dataset

In [0]:
#@title { display-mode: "form" }

dossier_TIPE = "/content/drive/My Drive/TIPE/"#@param {type:"string"}

Vérification des datasets

In [3]:
path = dossier_TIPE+"Galaxies_resized/"
dossiers = ["Train","Validation","Test"]
compte = []
for d in dossiers:
    compte.append(len(os.listdir(path+d)))
print([c/sum(compte) for c in compte])
print([c/10 for c in compte])

[0.6089108910891089, 0.297029702970297, 0.09405940594059406]
[24.6, 12.0, 3.8]


Création des classes de dataset Tensorflow

In [0]:
#D'après https://stackoverflow.com/questions/54590363/create-tensorflow-dataset-from-image-local-directory
class ArtificialDataset(tf.data.Dataset):
    def open_imgs(x):
        img = tf.io.read_file(x)
        img = tf.image.decode_jpeg(img)
        img = tf.image.convert_image_dtype(img, tf.float32)
        img = tf.image.resize(img, [256, 256])
        img = tf.clip_by_value(img,0,1)
        return img
        
    def _generator():
        for img_batch in ArtificialDataset.donnees:
            cleans = []
            for i,img in enumerate(img_batch):
                clean = ArtificialDataset.open_imgs(img)
                cleans.append(clean)
            cleans_t = tf.stack(cleans)
            yield (cleans_t)
    
    def __new__(cls,nom):
        ArtificialDataset.donnees = tf.data.Dataset.list_files(dossier_TIPE+"Galaxies_resized/"+nom+"/*.jpg").shuffle(len(os.listdir(dossier_TIPE+"Galaxies_resized/"+nom))).batch(10)
        #voir doc from_generator https://github.com/tensorflow/tensorflow/blob/f2b2563c6ce2001a117cd7adb36f303903e907ec/tensorflow/python/data/ops/dataset_ops.py#L669
        return tf.data.Dataset.from_generator(
            cls._generator,
            output_types=(tf.dtypes.float32),
            output_shapes=(tf.TensorShape([None,256,256,3])), #https://www.tensorflow.org/api_docs/python/tf/TensorShape?version=nightly
            args=None
        )

In [0]:

#Bruit gaussien origine : https://stackoverflow.com/questions/59286171/gaussian-blur-image-in-dataset-pipeline-in-tensorflow
def _gaussian_kernel(kernel_size, sigma, n_channels, dtype):
    """Génère le noyau de la couche de convolution du flou gaussien"""
    x = tf.range(-kernel_size // 2 + 1, kernel_size // 2 + 1, dtype=dtype)
    g = tf.math.exp(-(tf.pow(x, 2) / (2 * tf.pow(tf.cast(sigma, dtype), 2))))
    g_norm2d = tf.pow(tf.reduce_sum(g), 2)
    g_kernel = tf.tensordot(g, g, axes=0) / g_norm2d
    g_kernel = tf.expand_dims(g_kernel, axis=-1)
    return tf.expand_dims(tf.tile(g_kernel, (1, 1, n_channels)), axis=-1)

def apply_blur(img):
    """Applique le flou gaussien"""
    img = tf.reshape(img,[1,256,256,3])
    blur = _gaussian_kernel(3, 2, 3, img.dtype)
    img = tf.nn.depthwise_conv2d(img, blur, [1,1,1,1], 'SAME')
    return tf.reshape(img,[256,256,3])
def bruitage(img):
    #Bruitage
    img_noise = tf.image.adjust_saturation(img,.8)
    img_noise = apply_blur(img_noise)
    img_noise = tf.image.adjust_contrast(img_noise,tf.reduce_sum(tf.random.uniform(shape=(1,),dtype=tf.float32,minval=0.1,maxval=0.2)))
    ajout = 0.1
    img_noise = img_noise+tf.concat([tf.ones(shape=[256,256,1])*ajout,tf.zeros(shape=[256,256,2])],axis=-1)
    img_noise = tf.image.adjust_brightness(img_noise,-ajout)
    img_noise = tf.image.adjust_hue(img_noise,tf.reduce_sum(tf.random.uniform(shape=(1,),dtype=tf.float32,minval=0.06,maxval=0.09)))
    img_noise = tf.clip_by_value(img_noise,0,1)
    return img_noise
def traitement(img_batch):
    """Modification par image"""
    noised = tf.map_fn(bruitage,img_batch)
    img_noise_batch = tf.concat([tf.slice(noised,[0,0,0,0],[len(img_batch)//2,256,256,3]),tf.slice(img_batch,[len(img_batch)//2,0,0,0],[len(img_batch)-len(img_batch)//2,256,256,3])],axis=0)
    return img_batch,img_noise_batch

# Modèle

## Couche de convolution et déconvolution liée

In [0]:
class Conv2DTranspose_tied(Conv2DTranspose):
    def __init__(self,
               filters,
               kernel_size,
               strides=(1, 1),
               padding='valid',
               output_padding=None,
               data_format=None,
               dilation_rate=(1, 1),
               activation=None,
               use_bias=True,
               kernel_initializer='glorot_uniform',
               bias_initializer='zeros',
               kernel_regularizer=None,
               bias_regularizer=None,
               activity_regularizer=None,
               kernel_constraint=None,
               bias_constraint=None,
               tied_to = None,
               **kwargs):
        self.tied_to = tied_to
        self.initial_weights = tied_to.initial_weights
        super(Conv2DTranspose_tied, self).__init__(
            filters=filters,
            kernel_size=kernel_size,
            strides=strides,
            padding=padding,
            data_format=data_format,
            dilation_rate=dilation_rate,
            activation=activations.get(activation),
            use_bias=use_bias,
            kernel_initializer=initializers.get(kernel_initializer),
            bias_initializer=initializers.get(bias_initializer),
            kernel_regularizer=regularizers.get(kernel_regularizer),
            bias_regularizer=regularizers.get(bias_regularizer),
            activity_regularizer=regularizers.get(activity_regularizer),
            kernel_constraint=constraints.get(kernel_constraint),
            bias_constraint=constraints.get(bias_constraint),
            **kwargs)
    def build(self,input_shape):
        super(Conv2DTranspose_tied, self).build(self,input_shape)
        self.initial_weights = tied_to.initial_weights
    def call(self, inputs):
        self.kernel = tf.transpose(self.tied_to.kernel, (1, 0, 2, 3))
        if self.bias:
            if self.dim_ordering == 'th':
                output += K.reshape(self.b, (1, self.nb_filter, 1, 1))
            elif self.dim_ordering == 'tf':
                output += K.reshape(self.b, (1, 1, 1, self.nb_filter))
            else:
                raise Exception('Invalid dim_ordering: ' + self.dim_ordering)
        output = self.activation(output)
        return output
        if self._recreate_conv_op(inputs):
            self._convolution_op = nn_ops.Convolution(
                inputs.get_shape(),
                filter_shape=self.kernel.shape,
                dilation_rate=self.dilation_rate,
                strides=self.strides,
                padding=self._padding_op,
                data_format=self._conv_op_data_format)

        # Apply causal padding to inputs for Conv1D.
        if self.padding == 'causal' and self.__class__.__name__ == 'Conv1D':
            ts = array_ops.pad(inputs, self._compute_causal_padding())

        outputs = self._convolution_op(inputs, self.kernel)

        if self.use_bias:
            if self.data_format == 'channels_first':
                if self.rank == 1:
                    # nn.bias_add does not accept a 1D input tensor.
                    bias = array_ops.reshape(self.bias, (1, self.filters, 1))
                    outputs += bias
                else:
                    outputs = nn.bias_add(outputs, self.bias, data_format='NCHW')
        else:
            outputs = nn.bias_add(outputs, self.bias, data_format='NHWC')

        if self.activation is not None:
            return self.activation(outputs)
        return outputs

# Autoencoeur simple

On met en place de l'optimisation bayesienne pour former le reseau au mieux sachant que :

L'image fait $$256=2⁸$$ pixels 

un max pooling donnant $$output = \frac{output}{2}$$ avec un stride de 2 et un noyau de 2

une couche de déconvolution donne :


$$newSize = (size - 1)\cdot stride + k$$ sans padding











On peut donc avoir entre 0 et 8 réductions de taille

Donc 
- 0 --> 8 Pooling avec `strides=2, padding='VALID', kernel=2` 
- couches de convolution
    - en nombre libre de toute contrainte théoriquement mais `en pratique on cherchera à en minimiser le nombre
    - étant liée `obligatoirement à la couche suivante` et peut-être `à d'autres couche de même taille d'image`
    - `kernel` entre 1 et 3
    - nombre de `filtre` entre 1 et 500
- autant de deconvolution que de pooling
    - `kernel=2`
    - `stride=2`

L'idée est de faire un modèle pas trop compliqué pour commencer càd :
- sans batch normalisation


Avec 9 couches de convolution par niveau

`En mettant une pénalité sur le nombre total de paramètres suivant : `

$$loss = MSE+tanh(\frac{nb_{paramètres}}{tau})$$

tanh pour avoir une bijection de $\mathbb{R⁺}$ dans $[0,1]$

On dimensionne tau de telle sorte que pour 1 million de paramatres la pénalité soit maximale, pour 100 000 très forte et pour 1 000 faible à très faible : `tau= 70 000`

In [0]:
#{ display-mode: "form" }
nb_max_conv_par_etage = 9#@param {type:'number'}
possibilites_filtres = [1,3,10,50,100,500]#@param {type:'string'}

On commence avec keras-tuner donnant plus de possibilité (nombre variable de couches) puis on continuera à nombre de couche fixées pour visualiser sur le tensorboard

In [0]:
def loss_nb_parameters_penalty(nb_param,tau):
    def loss(y_true, y_pred):
        return tf.keras.losses.mean_squared_error(y_true, y_pred) + np.tanh(nb_param/tau)
    return loss
def modele(hp):
    init_time = time.time()
    couche_inpt = Input(shape=(256,256,3),dtype=tf.dtypes.float32,name='Entree_env10x256x256x3')
    couche = convolution(hp,0,couche_inpt,'desc')
    couche = couche_inpt
    nb_pool = hp.Int('nb_pool',min_value=0,max_value=8,default=0)
    for index_pooling in range(nb_pool):
        type_pool = hp.Choice('pool_type_%d'%index_pooling,['avg','max'],default='max')
        if type_pool == 'avg':
            couche = AveragePooling2D(pool_size=2,strides=2,padding='VALID',name='AveragePool_etg_%d'%(index_pooling))(couche)
        else:
            couche = MaxPooling2D(pool_size=2,strides=2,padding='VALID',name='MaxPool_etg_%d'%(index_pooling))(couche)
        couche = convolution(hp,index_pooling+1,couche,'desc')
    for index_deconv in range(nb_pool):
        filters = hp.Choice('filtre_deconv_etg_%d_%s'%(index_deconv,'remont'),possibilites_filtres,default=1)
        couche = Conv2DTranspose(filters=filters,kernel_size=2,strides=2,name='TransposedConv_Deconv_etg_%d_f%d'%(filters,index_deconv))(couche)
        couche = convolution(hp,index_deconv,couche,'remont')
    couche = Conv2D(filters=3,kernel_size=2,padding='SAME',name='Convolution_fin_k%d_f%d_activ_%s'%(3,2,'linear'))(couche)
    model = Model(inputs=couche_inpt,outputs=couche,name='Debruiteur')
    print("Nb param : ",model.count_params())
    model.compile(optimizer='sgd',#Adam(hp.Choice('learning_rate', [1e-2, 1e-3, 1e-4,1e-5,1e-6],default=1e-4),
                                                     #beta_1=hp.Float('beta_1',8e-1,1.2,default=0.99),
                                                     #beta_2=hp.Float('beta_2',8e-1,1.2,default=0.99)),
                  loss=loss_nb_parameters_penalty(model.count_params(),7e4),metrics=['accuracy'])
    return model
def convolution(hp,etage,inpt,pool_txt):
    couche = inpt
    L_conv_param = []
    nb_conv = hp.Int('nb_conv_etg_%d_%s'%(etage,pool_txt),min_value=0,max_value=nb_max_conv_par_etage+1,default=1)
    for index_conv in range(nb_conv):
        filters = hp.Choice('filtre_conv_index_%d_etg_%d_%s'%(index_conv,etage,pool_txt),possibilites_filtres,default=1)
        kernel = hp.Choice('kernel_conv_index_%d_etg_%d_%s'%(index_conv,etage,pool_txt),[2,3],default=2)
        activ = hp.Choice("activation_conv_index_%d_etg_%d_%s"%(index_conv,etage,pool_txt),['linear','relu','elu','selu','tanh'],default='relu')
        couche = Conv2D(filters=filters,
                             kernel_size=kernel,
                             padding='SAME',
                             activation=activ,
                             name='Convolution_%s_etg_%d_id_%d_k%d_f%d_activ_%s'%(pool_txt,etage,index_conv,filters,kernel,activ))(couche)
    return couche

In [9]:
tuner = BayesianOptimization(
    modele,
    objective='accuracy',
    max_trials=25,
    executions_per_trial=3,
    directory='Bayesian_optimization',
    project_name='Modele000002_Keras_Bayesian_9conv_penaliteNbSGD2'
)
# tuner.search_space_summary()

INFO:tensorflow:Reloading Oracle from existing project Bayesian_optimization/Modele000002_Keras_Bayesian_9conv_penaliteNbSGD2/oracle.json
Nb param :  39
INFO:tensorflow:Reloading Tuner from Bayesian_optimization/Modele000002_Keras_Bayesian_9conv_penaliteNbSGD2/tuner0.json


In [0]:
dataset_train = ArtificialDataset(nom="Train").map(traitement,num_parallel_calls=tf.data.experimental.AUTOTUNE).cache().prefetch(tf.data.experimental.AUTOTUNE)
dataset_val = ArtificialDataset(nom="Validation").map(traitement,num_parallel_calls=tf.data.experimental.AUTOTUNE).cache().prefetch(tf.data.experimental.AUTOTUNE)

# tuner.search(dataset_train,
#              epochs=5,
#              validation_data=dataset_val,
#              callbacks=[tf.keras.callbacks.EarlyStopping(min_delta=1e-3)])


In [11]:
tuner.results_summary()

In [12]:
[model] = tuner.get_best_models()
print(model.summary())
print([l.name for l in model.layers])

Nb param :  1005199
Model: "Debruiteur"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
Entree_env10x256x256x3 (Inpu [(None, 256, 256, 3)]     0         
_________________________________________________________________
AveragePool_etg_0 (AveragePo (None, 128, 128, 3)       0         
_________________________________________________________________
MaxPool_etg_1 (MaxPooling2D) (None, 64, 64, 3)         0         
_________________________________________________________________
AveragePool_etg_2 (AveragePo (None, 32, 32, 3)         0         
_________________________________________________________________
Convolution_desc_etg_3_id_0_ (None, 32, 32, 1)         13        
_________________________________________________________________
Convolution_desc_etg_3_id_1_ (None, 32, 32, 1)         5         
_________________________________________________________________
Convolution_desc_etg_3_id_2_ (None, 

In [13]:
dossier = "./Bayesian_optimization/Modele000002_Keras_Bayesian_9conv_penaliteNbSGD2/Top1Explain"#@param {type:'string'}
!rm -rf $dossier
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=dossier, 
                                                      histogram_freq=1,
                                                      write_images=True)

model.fit(dataset_train,epochs=3, callbacks=[tensorboard_callback], validation_data=dataset_val)
%reload_ext tensorboard
%tensorboard --logdir $dossier

Epoch 1/3
12/12 [==============================] - 89s 7s/step - loss: 1.0071 - accuracy: 0.8400 - val_loss: 1.0062 - val_accuracy: 0.8443
Epoch 2/3
12/12 [==============================] - 7s 569ms/step - loss: 1.0067 - accuracy: 0.8400 - val_loss: 1.0058 - val_accuracy: 0.8443
Epoch 3/3
12/12 [==============================] - 9s 711ms/step - loss: 1.0063 - accuracy: 0.8400 - val_loss: 1.0055 - val_accuracy: 0.8443


ERROR: Failed to launch TensorBoard (exited with 1).
Contents of stderr:
Traceback (most recent call last):
  File "/usr/local/bin/tensorboard", line 8, in <module>
    sys.exit(run_main())
  File "/usr/local/lib/python3.6/dist-packages/tensorboard/main.py", line 75, in run_main
    app.run(tensorboard.main, flags_parser=tensorboard.configure)
  File "/usr/local/lib/python3.6/dist-packages/absl/app.py", line 299, in run
    _run_main(main, args)
  File "/usr/local/lib/python3.6/dist-packages/absl/app.py", line 250, in _run_main
    sys.exit(main(argv))
  File "/usr/local/lib/python3.6/dist-packages/tensorboard/program.py", line 289, in main
    return runner(self.flags) or 0
  File "/usr/local/lib/python3.6/dist-packages/tensorboard/program.py", line 305, in _run_serve_subcommand
    server = self._make_server()
  File "/usr/local/lib/python3.6/dist-packages/tensorboard/program.py", line 409, in _make_server
    self.flags, self.plugin_loaders, self.assets_zip_provider
  File "/usr/loc

# Résultats premier groupe d'entrainement (avant erreur de dimension)

Results summary
|-Results in Bayesian_optimization/Modele000002_Keras_Bayesian
|-Showing 10 best trials
|-Objective(name='val_accuracy', direction='max')
Trial summary
|-Trial ID: e3de1ef4d6472fd0557549403c1a956e
|-Score: 0.846445620059967
|-Best step: 0
Hyperparameters:
|-activation_conv_index_0_etg_0_desc: relu
|-activation_conv_index_0_etg_0_remont: relu
|-activation_conv_index_0_etg_1_desc: linear
|-activation_conv_index_0_etg_1_remont: linear
|-activation_conv_index_0_etg_2_desc: selu
|-activation_conv_index_0_etg_2_remont: relu
|-activation_conv_index_0_etg_3_desc: linear
|-activation_conv_index_0_etg_3_remont: relu
|-activation_conv_index_0_etg_4_desc: elu
|-activation_conv_index_0_etg_4_remont: relu
|-activation_conv_index_0_etg_5_desc: relu
|-activation_conv_index_0_etg_5_remont: relu
|-activation_conv_index_0_etg_6_desc: relu
|-activation_conv_index_0_etg_6_remont: relu
|-activation_conv_index_0_etg_7_desc: relu
|-activation_conv_index_0_etg_7_remont: relu
|-activation_conv_index_0_etg_8_desc: relu
|-activation_conv_index_1_etg_0_desc: selu
|-activation_conv_index_1_etg_0_remont: relu
|-activation_conv_index_1_etg_1_desc: linear
|-activation_conv_index_1_etg_1_remont: relu
|-activation_conv_index_1_etg_2_desc: relu
|-activation_conv_index_1_etg_2_remont: elu
|-activation_conv_index_1_etg_3_desc: relu
|-activation_conv_index_1_etg_3_remont: relu
|-activation_conv_index_2_etg_0_desc: selu
|-activation_conv_index_2_etg_0_remont: elu
|-activation_conv_index_2_etg_1_desc: tanh
|-activation_conv_index_2_etg_2_desc: relu
|-activation_conv_index_2_etg_2_remont: tanh
|-activation_conv_index_2_etg_3_desc: relu
|-activation_conv_index_2_etg_3_remont: relu
|-activation_conv_index_3_etg_0_remont: selu
|-activation_conv_index_3_etg_2_desc: relu
|-beta_1: 1.1567454740840701
|-beta_2: 1.1580270355932856
|-filtre_conv_index_0_etg_0_desc: 100
|-filtre_conv_index_0_etg_0_remont: 100
|-filtre_conv_index_0_etg_1_desc: 1
|-filtre_conv_index_0_etg_1_remont: 100
|-filtre_conv_index_0_etg_2_desc: 10
|-filtre_conv_index_0_etg_2_remont: 3
|-filtre_conv_index_0_etg_3_desc: 3
|-filtre_conv_index_0_etg_3_remont: 50
|-filtre_conv_index_0_etg_4_desc: 500
|-filtre_conv_index_0_etg_4_remont: 1
|-filtre_conv_index_0_etg_5_desc: 1
|-filtre_conv_index_0_etg_5_remont: 1
|-filtre_conv_index_0_etg_6_desc: 1
|-filtre_conv_index_0_etg_6_remont: 1
|-filtre_conv_index_0_etg_7_desc: 1
|-filtre_conv_index_0_etg_7_remont: 1
|-filtre_conv_index_0_etg_8_desc: 1
|-filtre_conv_index_1_etg_0_desc: 50
|-filtre_conv_index_1_etg_0_remont: 50
|-filtre_conv_index_1_etg_1_desc: 500
|-filtre_conv_index_1_etg_1_remont: 1
|-filtre_conv_index_1_etg_2_desc: 1
|-filtre_conv_index_1_etg_2_remont: 50
|-filtre_conv_index_1_etg_3_desc: 1
|-filtre_conv_index_1_etg_3_remont: 1
|-filtre_conv_index_2_etg_0_desc: 500
|-filtre_conv_index_2_etg_0_remont: 50
|-filtre_conv_index_2_etg_1_desc: 10
|-filtre_conv_index_2_etg_2_desc: 1
|-filtre_conv_index_2_etg_2_remont: 100
|-filtre_conv_index_2_etg_3_desc: 1
|-filtre_conv_index_2_etg_3_remont: 1
|-filtre_conv_index_3_etg_0_remont: 3
|-filtre_conv_index_3_etg_2_desc: 1
|-filtre_deconv_etg_0_remont: 1
|-filtre_deconv_etg_1_remont: 3
|-filtre_deconv_etg_2_remont: 50
|-filtre_deconv_etg_3_remont: 50
|-filtre_deconv_etg_4_remont: 1
|-filtre_deconv_etg_5_remont: 1
|-filtre_deconv_etg_6_remont: 1
|-filtre_deconv_etg_7_remont: 1
|-kernel_conv_index_0_etg_0_desc: 3
|-kernel_conv_index_0_etg_0_remont: 2
|-kernel_conv_index_0_etg_1_desc: 3
|-kernel_conv_index_0_etg_1_remont: 3
|-kernel_conv_index_0_etg_2_desc: 3
|-kernel_conv_index_0_etg_2_remont: 2
|-kernel_conv_index_0_etg_3_desc: 2
|-kernel_conv_index_0_etg_3_remont: 2
|-kernel_conv_index_0_etg_4_desc: 3
|-kernel_conv_index_0_etg_4_remont: 2
|-kernel_conv_index_0_etg_5_desc: 2
|-kernel_conv_index_0_etg_5_remont: 2
|-kernel_conv_index_0_etg_6_desc: 2
|-kernel_conv_index_0_etg_6_remont: 2
|-kernel_conv_index_0_etg_7_desc: 2
|-kernel_conv_index_0_etg_7_remont: 2
|-kernel_conv_index_0_etg_8_desc: 2
|-kernel_conv_index_1_etg_0_desc: 3
|-kernel_conv_index_1_etg_0_remont: 3
|-kernel_conv_index_1_etg_1_desc: 2
|-kernel_conv_index_1_etg_1_remont: 2
|-kernel_conv_index_1_etg_2_desc: 2
|-kernel_conv_index_1_etg_2_remont: 2
|-kernel_conv_index_1_etg_3_desc: 2
|-kernel_conv_index_1_etg_3_remont: 2
|-kernel_conv_index_2_etg_0_desc: 3
|-kernel_conv_index_2_etg_0_remont: 3
|-kernel_conv_index_2_etg_1_desc: 2
|-kernel_conv_index_2_etg_2_desc: 2
|-kernel_conv_index_2_etg_2_remont: 3
|-kernel_conv_index_2_etg_3_desc: 2
|-kernel_conv_index_2_etg_3_remont: 2
|-kernel_conv_index_3_etg_0_remont: 2
|-kernel_conv_index_3_etg_2_desc: 2
|-learning_rate: 1e-05
|-nb_conv_etg_0_desc: 3
|-nb_conv_etg_0_remont: 4
|-nb_conv_etg_1_desc: 2
|-nb_conv_etg_1_remont: 2
|-nb_conv_etg_2_desc: 4
|-nb_conv_etg_2_remont: 2
|-nb_conv_etg_3_desc: 3
|-nb_conv_etg_3_remont: 3
|-nb_conv_etg_4_desc: 0
|-nb_conv_etg_4_remont: 1
|-nb_conv_etg_5_desc: 1
|-nb_conv_etg_5_remont: 1
|-nb_conv_etg_6_desc: 1
|-nb_conv_etg_6_remont: 1
|-nb_conv_etg_7_desc: 1
|-nb_conv_etg_7_remont: 1
|-nb_conv_etg_8_desc: 1
|-nb_pool: 8
|-pool_type_0: max
|-pool_type_1: max
|-pool_type_2: max
|-pool_type_3: max
|-pool_type_4: max
|-pool_type_5: max
|-pool_type_6: max
|-pool_type_7: max
Trial summary
|-Trial ID: f5fa2f79bdd80f70c674000201c6f0e0
|-Score: 0.846445620059967
|-Best step: 0
Hyperparameters:
|-activation_conv_index_0_etg_0_desc: tanh
|-activation_conv_index_0_etg_0_remont: selu
|-activation_conv_index_0_etg_1_desc: tanh
|-activation_conv_index_0_etg_1_remont: relu
|-activation_conv_index_0_etg_2_desc: tanh
|-activation_conv_index_0_etg_2_remont: elu
|-activation_conv_index_0_etg_3_desc: linear
|-activation_conv_index_0_etg_3_remont: selu
|-activation_conv_index_0_etg_4_desc: tanh
|-activation_conv_index_0_etg_4_remont: tanh
|-activation_conv_index_0_etg_5_desc: selu
|-activation_conv_index_0_etg_5_remont: elu
|-activation_conv_index_0_etg_6_desc: linear
|-activation_conv_index_0_etg_6_remont: linear
|-activation_conv_index_0_etg_7_desc: selu
|-activation_conv_index_0_etg_7_remont: tanh
|-activation_conv_index_0_etg_8_desc: relu
|-activation_conv_index_1_etg_0_desc: tanh
|-activation_conv_index_1_etg_0_remont: elu
|-activation_conv_index_1_etg_1_desc: tanh
|-activation_conv_index_1_etg_1_remont: selu
|-activation_conv_index_1_etg_2_desc: linear
|-activation_conv_index_1_etg_2_remont: elu
|-activation_conv_index_1_etg_3_desc: linear
|-activation_conv_index_1_etg_3_remont: elu
|-activation_conv_index_1_etg_4_desc: relu
|-activation_conv_index_1_etg_4_remont: relu
|-activation_conv_index_1_etg_5_remont: relu
|-activation_conv_index_1_etg_6_desc: relu
|-activation_conv_index_1_etg_7_desc: relu
|-activation_conv_index_1_etg_7_remont: relu
|-activation_conv_index_1_etg_8_desc: relu
|-activation_conv_index_2_etg_0_desc: relu
|-activation_conv_index_2_etg_0_remont: linear
|-activation_conv_index_2_etg_1_desc: selu
|-activation_conv_index_2_etg_2_desc: elu
|-activation_conv_index_2_etg_2_remont: selu
|-activation_conv_index_2_etg_3_desc: linear
|-activation_conv_index_2_etg_3_remont: relu
|-activation_conv_index_2_etg_4_desc: relu
|-activation_conv_index_2_etg_4_remont: relu
|-activation_conv_index_2_etg_7_desc: relu
|-activation_conv_index_2_etg_7_remont: relu
|-activation_conv_index_2_etg_8_desc: relu
|-activation_conv_index_3_etg_0_remont: elu
|-activation_conv_index_3_etg_2_desc: relu
|-activation_conv_index_3_etg_7_remont: relu
|-beta_1: 0.9672309653263195
|-beta_2: 1.0901985450142682
|-filtre_conv_index_0_etg_0_desc: 100
|-filtre_conv_index_0_etg_0_remont: 100
|-filtre_conv_index_0_etg_1_desc: 10
|-filtre_conv_index_0_etg_1_remont: 3
|-filtre_conv_index_0_etg_2_desc: 10
|-filtre_conv_index_0_etg_2_remont: 500
|-filtre_conv_index_0_etg_3_desc: 50
|-filtre_conv_index_0_etg_3_remont: 100
|-filtre_conv_index_0_etg_4_desc: 1
|-filtre_conv_index_0_etg_4_remont: 1
|-filtre_conv_index_0_etg_5_desc: 10
|-filtre_conv_index_0_etg_5_remont: 100
|-filtre_conv_index_0_etg_6_desc: 500
|-filtre_conv_index_0_etg_6_remont: 50
|-filtre_conv_index_0_etg_7_desc: 500
|-filtre_conv_index_0_etg_7_remont: 100
|-filtre_conv_index_0_etg_8_desc: 500
|-filtre_conv_index_1_etg_0_desc: 50
|-filtre_conv_index_1_etg_0_remont: 50
|-filtre_conv_index_1_etg_1_desc: 500
|-filtre_conv_index_1_etg_1_remont: 3
|-filtre_conv_index_1_etg_2_desc: 500
|-filtre_conv_index_1_etg_2_remont: 3
|-filtre_conv_index_1_etg_3_desc: 500
|-filtre_conv_index_1_etg_3_remont: 50
|-filtre_conv_index_1_etg_4_desc: 1
|-filtre_conv_index_1_etg_4_remont: 1
|-filtre_conv_index_1_etg_5_remont: 1
|-filtre_conv_index_1_etg_6_desc: 1
|-filtre_conv_index_1_etg_7_desc: 1
|-filtre_conv_index_1_etg_7_remont: 1
|-filtre_conv_index_1_etg_8_desc: 1
|-filtre_conv_index_2_etg_0_desc: 3
|-filtre_conv_index_2_etg_0_remont: 10
|-filtre_conv_index_2_etg_1_desc: 100
|-filtre_conv_index_2_etg_2_desc: 50
|-filtre_conv_index_2_etg_2_remont: 50
|-filtre_conv_index_2_etg_3_desc: 500
|-filtre_conv_index_2_etg_3_remont: 50
|-filtre_conv_index_2_etg_4_desc: 1
|-filtre_conv_index_2_etg_4_remont: 1
|-filtre_conv_index_2_etg_7_desc: 1
|-filtre_conv_index_2_etg_7_remont: 1
|-filtre_conv_index_2_etg_8_desc: 1
|-filtre_conv_index_3_etg_0_remont: 3
|-filtre_conv_index_3_etg_2_desc: 1
|-filtre_conv_index_3_etg_7_remont: 1
|-filtre_deconv_etg_0_remont: 1
|-filtre_deconv_etg_1_remont: 50
|-filtre_deconv_etg_2_remont: 500
|-filtre_deconv_etg_3_remont: 1
|-filtre_deconv_etg_4_remont: 3
|-filtre_deconv_etg_5_remont: 50
|-filtre_deconv_etg_6_remont: 10
|-filtre_deconv_etg_7_remont: 100
|-kernel_conv_index_0_etg_0_desc: 2
|-kernel_conv_index_0_etg_0_remont: 3
|-kernel_conv_index_0_etg_1_desc: 3
|-kernel_conv_index_0_etg_1_remont: 3
|-kernel_conv_index_0_etg_2_desc: 3
|-kernel_conv_index_0_etg_2_remont: 3
|-kernel_conv_index_0_etg_3_desc: 2
|-kernel_conv_index_0_etg_3_remont: 3
|-kernel_conv_index_0_etg_4_desc: 3
|-kernel_conv_index_0_etg_4_remont: 2
|-kernel_conv_index_0_etg_5_desc: 2
|-kernel_conv_index_0_etg_5_remont: 3
|-kernel_conv_index_0_etg_6_desc: 2
|-kernel_conv_index_0_etg_6_remont: 2
|-kernel_conv_index_0_etg_7_desc: 2
|-kernel_conv_index_0_etg_7_remont: 3
|-kernel_conv_index_0_etg_8_desc: 2
|-kernel_conv_index_1_etg_0_desc: 3
|-kernel_conv_index_1_etg_0_remont: 2
|-kernel_conv_index_1_etg_1_desc: 3
|-kernel_conv_index_1_etg_1_remont: 3
|-kernel_conv_index_1_etg_2_desc: 2
|-kernel_conv_index_1_etg_2_remont: 3
|-kernel_conv_index_1_etg_3_desc: 2
|-kernel_conv_index_1_etg_3_remont: 2
|-kernel_conv_index_1_etg_4_desc: 2
|-kernel_conv_index_1_etg_4_remont: 2
|-kernel_conv_index_1_etg_5_remont: 2
|-kernel_conv_index_1_etg_6_desc: 2
|-kernel_conv_index_1_etg_7_desc: 2
|-kernel_conv_index_1_etg_7_remont: 2
|-kernel_conv_index_1_etg_8_desc: 2
|-kernel_conv_index_2_etg_0_desc: 3
|-kernel_conv_index_2_etg_0_remont: 3
|-kernel_conv_index_2_etg_1_desc: 2
|-kernel_conv_index_2_etg_2_desc: 3
|-kernel_conv_index_2_etg_2_remont: 2
|-kernel_conv_index_2_etg_3_desc: 3
|-kernel_conv_index_2_etg_3_remont: 3
|-kernel_conv_index_2_etg_4_desc: 2
|-kernel_conv_index_2_etg_4_remont: 2
|-kernel_conv_index_2_etg_7_desc: 2
|-kernel_conv_index_2_etg_7_remont: 2
|-kernel_conv_index_2_etg_8_desc: 2
|-kernel_conv_index_3_etg_0_remont: 3
|-kernel_conv_index_3_etg_2_desc: 3
|-kernel_conv_index_3_etg_7_remont: 2
|-learning_rate: 1e-05
|-nb_conv_etg_0_desc: 3
|-nb_conv_etg_0_remont: 1
|-nb_conv_etg_1_desc: 1
|-nb_conv_etg_1_remont: 0
|-nb_conv_etg_2_desc: 2
|-nb_conv_etg_2_remont: 2
|-nb_conv_etg_3_desc: 1
|-nb_conv_etg_3_remont: 1
|-nb_conv_etg_4_desc: 3
|-nb_conv_etg_4_remont: 3
|-nb_conv_etg_5_desc: 1
|-nb_conv_etg_5_remont: 2
|-nb_conv_etg_6_desc: 2
|-nb_conv_etg_6_remont: 0
|-nb_conv_etg_7_desc: 3
|-nb_conv_etg_7_remont: 4
|-nb_conv_etg_8_desc: 3
|-nb_pool: 8
|-pool_type_0: avg
|-pool_type_1: avg
|-pool_type_2: max
|-pool_type_3: max
|-pool_type_4: avg
|-pool_type_5: avg
|-pool_type_6: avg
|-pool_type_7: avg
Trial summary
|-Trial ID: dd92e50cfa43158b2b257e19be3fa684
|-Score: 0.8454128305117289
|-Best step: 0
Hyperparameters:
|-activation_conv_index_0_etg_0_desc: tanh
|-activation_conv_index_0_etg_0_remont: selu
|-activation_conv_index_0_etg_1_desc: tanh
|-activation_conv_index_0_etg_1_remont: linear
|-activation_conv_index_0_etg_2_desc: linear
|-activation_conv_index_0_etg_2_remont: selu
|-activation_conv_index_0_etg_3_desc: tanh
|-activation_conv_index_0_etg_3_remont: elu
|-activation_conv_index_0_etg_4_desc: relu
|-activation_conv_index_0_etg_4_remont: selu
|-activation_conv_index_0_etg_5_desc: relu
|-activation_conv_index_0_etg_5_remont: relu
|-activation_conv_index_0_etg_6_desc: relu
|-activation_conv_index_0_etg_6_remont: relu
|-activation_conv_index_0_etg_7_desc: relu
|-activation_conv_index_0_etg_7_remont: selu
|-activation_conv_index_0_etg_8_desc: relu
|-activation_conv_index_1_etg_0_desc: tanh
|-activation_conv_index_1_etg_0_remont: tanh
|-activation_conv_index_1_etg_1_desc: elu
|-activation_conv_index_1_etg_1_remont: selu
|-activation_conv_index_1_etg_2_desc: selu
|-activation_conv_index_1_etg_2_remont: selu
|-activation_conv_index_1_etg_3_desc: linear
|-activation_conv_index_1_etg_3_remont: tanh
|-activation_conv_index_1_etg_4_desc: selu
|-activation_conv_index_1_etg_4_remont: linear
|-activation_conv_index_1_etg_5_desc: linear
|-activation_conv_index_1_etg_5_remont: selu
|-activation_conv_index_1_etg_6_desc: relu
|-activation_conv_index_1_etg_6_remont: linear
|-activation_conv_index_1_etg_7_desc: linear
|-activation_conv_index_1_etg_7_remont: relu
|-activation_conv_index_1_etg_8_desc: elu
|-activation_conv_index_2_etg_0_desc: linear
|-activation_conv_index_2_etg_0_remont: linear
|-activation_conv_index_2_etg_1_desc: selu
|-activation_conv_index_2_etg_1_remont: relu
|-activation_conv_index_2_etg_2_desc: tanh
|-activation_conv_index_2_etg_2_remont: linear
|-activation_conv_index_2_etg_3_desc: linear
|-activation_conv_index_2_etg_3_remont: relu
|-activation_conv_index_2_etg_4_desc: selu
|-activation_conv_index_2_etg_4_remont: selu
|-activation_conv_index_2_etg_5_remont: selu
|-activation_conv_index_2_etg_7_desc: linear
|-activation_conv_index_2_etg_7_remont: elu
|-activation_conv_index_2_etg_8_desc: relu
|-activation_conv_index_3_etg_0_desc: relu
|-activation_conv_index_3_etg_0_remont: elu
|-activation_conv_index_3_etg_1_desc: relu
|-activation_conv_index_3_etg_1_remont: selu
|-activation_conv_index_3_etg_2_desc: relu
|-activation_conv_index_3_etg_4_desc: selu
|-activation_conv_index_3_etg_7_remont: elu
|-beta_1: 0.9237723724948255
|-beta_2: 0.8373641141314946
|-filtre_conv_index_0_etg_0_desc: 10
|-filtre_conv_index_0_etg_0_remont: 1
|-filtre_conv_index_0_etg_1_desc: 500
|-filtre_conv_index_0_etg_1_remont: 50
|-filtre_conv_index_0_etg_2_desc: 1
|-filtre_conv_index_0_etg_2_remont: 3
|-filtre_conv_index_0_etg_3_desc: 500
|-filtre_conv_index_0_etg_3_remont: 100
|-filtre_conv_index_0_etg_4_desc: 100
|-filtre_conv_index_0_etg_4_remont: 1
|-filtre_conv_index_0_etg_5_desc: 1
|-filtre_conv_index_0_etg_5_remont: 50
|-filtre_conv_index_0_etg_6_desc: 100
|-filtre_conv_index_0_etg_6_remont: 1
|-filtre_conv_index_0_etg_7_desc: 50
|-filtre_conv_index_0_etg_7_remont: 500
|-filtre_conv_index_0_etg_8_desc: 100
|-filtre_conv_index_1_etg_0_desc: 3
|-filtre_conv_index_1_etg_0_remont: 100
|-filtre_conv_index_1_etg_1_desc: 50
|-filtre_conv_index_1_etg_1_remont: 50
|-filtre_conv_index_1_etg_2_desc: 10
|-filtre_conv_index_1_etg_2_remont: 1
|-filtre_conv_index_1_etg_3_desc: 10
|-filtre_conv_index_1_etg_3_remont: 10
|-filtre_conv_index_1_etg_4_desc: 500
|-filtre_conv_index_1_etg_4_remont: 500
|-filtre_conv_index_1_etg_5_desc: 500
|-filtre_conv_index_1_etg_5_remont: 1
|-filtre_conv_index_1_etg_6_desc: 100
|-filtre_conv_index_1_etg_6_remont: 50
|-filtre_conv_index_1_etg_7_desc: 50
|-filtre_conv_index_1_etg_7_remont: 3
|-filtre_conv_index_1_etg_8_desc: 50
|-filtre_conv_index_2_etg_0_desc: 50
|-filtre_conv_index_2_etg_0_remont: 3
|-filtre_conv_index_2_etg_1_desc: 10
|-filtre_conv_index_2_etg_1_remont: 100
|-filtre_conv_index_2_etg_2_desc: 50
|-filtre_conv_index_2_etg_2_remont: 100
|-filtre_conv_index_2_etg_3_desc: 500
|-filtre_conv_index_2_etg_3_remont: 1
|-filtre_conv_index_2_etg_4_desc: 100
|-filtre_conv_index_2_etg_4_remont: 100
|-filtre_conv_index_2_etg_5_remont: 10
|-filtre_conv_index_2_etg_7_desc: 3
|-filtre_conv_index_2_etg_7_remont: 500
|-filtre_conv_index_2_etg_8_desc: 10
|-filtre_conv_index_3_etg_0_desc: 50
|-filtre_conv_index_3_etg_0_remont: 50
|-filtre_conv_index_3_etg_1_desc: 50
|-filtre_conv_index_3_etg_1_remont: 100
|-filtre_conv_index_3_etg_2_desc: 10
|-filtre_conv_index_3_etg_4_desc: 10
|-filtre_conv_index_3_etg_7_remont: 1
|-filtre_deconv_etg_0_remont: 500
|-filtre_deconv_etg_1_remont: 10
|-filtre_deconv_etg_2_remont: 500
|-filtre_deconv_etg_3_remont: 100
|-filtre_deconv_etg_4_remont: 500
|-filtre_deconv_etg_5_remont: 50
|-filtre_deconv_etg_6_remont: 3
|-filtre_deconv_etg_7_remont: 1
|-kernel_conv_index_0_etg_0_desc: 2
|-kernel_conv_index_0_etg_0_remont: 3
|-kernel_conv_index_0_etg_1_desc: 2
|-kernel_conv_index_0_etg_1_remont: 2
|-kernel_conv_index_0_etg_2_desc: 2
|-kernel_conv_index_0_etg_2_remont: 3
|-kernel_conv_index_0_etg_3_desc: 3
|-kernel_conv_index_0_etg_3_remont: 2
|-kernel_conv_index_0_etg_4_desc: 3
|-kernel_conv_index_0_etg_4_remont: 2
|-kernel_conv_index_0_etg_5_desc: 3
|-kernel_conv_index_0_etg_5_remont: 3
|-kernel_conv_index_0_etg_6_desc: 3
|-kernel_conv_index_0_etg_6_remont: 3
|-kernel_conv_index_0_etg_7_desc: 3
|-kernel_conv_index_0_etg_7_remont: 3
|-kernel_conv_index_0_etg_8_desc: 3
|-kernel_conv_index_1_etg_0_desc: 2
|-kernel_conv_index_1_etg_0_remont: 3
|-kernel_conv_index_1_etg_1_desc: 3
|-kernel_conv_index_1_etg_1_remont: 3
|-kernel_conv_index_1_etg_2_desc: 3
|-kernel_conv_index_1_etg_2_remont: 3
|-kernel_conv_index_1_etg_3_desc: 2
|-kernel_conv_index_1_etg_3_remont: 3
|-kernel_conv_index_1_etg_4_desc: 3
|-kernel_conv_index_1_etg_4_remont: 3
|-kernel_conv_index_1_etg_5_desc: 3
|-kernel_conv_index_1_etg_5_remont: 2
|-kernel_conv_index_1_etg_6_desc: 3
|-kernel_conv_index_1_etg_6_remont: 2
|-kernel_conv_index_1_etg_7_desc: 2
|-kernel_conv_index_1_etg_7_remont: 2
|-kernel_conv_index_1_etg_8_desc: 3
|-kernel_conv_index_2_etg_0_desc: 3
|-kernel_conv_index_2_etg_0_remont: 3
|-kernel_conv_index_2_etg_1_desc: 3
|-kernel_conv_index_2_etg_1_remont: 2
|-kernel_conv_index_2_etg_2_desc: 3
|-kernel_conv_index_2_etg_2_remont: 3
|-kernel_conv_index_2_etg_3_desc: 2
|-kernel_conv_index_2_etg_3_remont: 3
|-kernel_conv_index_2_etg_4_desc: 3
|-kernel_conv_index_2_etg_4_remont: 3
|-kernel_conv_index_2_etg_5_remont: 3
|-kernel_conv_index_2_etg_7_desc: 2
|-kernel_conv_index_2_etg_7_remont: 2
|-kernel_conv_index_2_etg_8_desc: 2
|-kernel_conv_index_3_etg_0_desc: 2
|-kernel_conv_index_3_etg_0_remont: 2
|-kernel_conv_index_3_etg_1_desc: 2
|-kernel_conv_index_3_etg_1_remont: 3
|-kernel_conv_index_3_etg_2_desc: 2
|-kernel_conv_index_3_etg_4_desc: 3
|-kernel_conv_index_3_etg_7_remont: 2
|-learning_rate: 0.001
|-nb_conv_etg_0_desc: 1
|-nb_conv_etg_0_remont: 4
|-nb_conv_etg_1_desc: 1
|-nb_conv_etg_1_remont: 2
|-nb_conv_etg_2_desc: 1
|-nb_conv_etg_2_remont: 2
|-nb_conv_etg_3_desc: 0
|-nb_conv_etg_3_remont: 3
|-nb_conv_etg_4_desc: 2
|-nb_conv_etg_4_remont: 0
|-nb_conv_etg_5_desc: 3
|-nb_conv_etg_5_remont: 2
|-nb_conv_etg_6_desc: 3
|-nb_conv_etg_6_remont: 3
|-nb_conv_etg_7_desc: 1
|-nb_conv_etg_7_remont: 0
|-nb_conv_etg_8_desc: 2
|-nb_pool: 2
|-pool_type_0: max
|-pool_type_1: max
|-pool_type_2: max
|-pool_type_3: avg
|-pool_type_4: max
|-pool_type_5: avg
|-pool_type_6: max
|-pool_type_7: max
Trial summary
|-Trial ID: 5816df68396ef8e46cacd3f962aadd2d
|-Score: 0.5878276079893112
|-Best step: 0
Hyperparameters:
|-activation_conv_index_0_etg_0_desc: elu
|-activation_conv_index_0_etg_0_remont: tanh
|-activation_conv_index_0_etg_1_desc: elu
|-activation_conv_index_0_etg_1_remont: tanh
|-activation_conv_index_0_etg_2_desc: tanh
|-activation_conv_index_0_etg_2_remont: linear
|-activation_conv_index_0_etg_3_desc: selu
|-activation_conv_index_0_etg_3_remont: elu
|-activation_conv_index_0_etg_4_desc: elu
|-activation_conv_index_0_etg_4_remont: relu
|-activation_conv_index_0_etg_5_desc: tanh
|-activation_conv_index_0_etg_5_remont: relu
|-activation_conv_index_0_etg_6_desc: relu
|-activation_conv_index_0_etg_6_remont: tanh
|-activation_conv_index_0_etg_7_desc: tanh
|-activation_conv_index_0_etg_7_remont: selu
|-activation_conv_index_0_etg_8_desc: relu
|-activation_conv_index_1_etg_0_desc: linear
|-activation_conv_index_1_etg_0_remont: linear
|-activation_conv_index_1_etg_1_desc: selu
|-activation_conv_index_1_etg_1_remont: tanh
|-activation_conv_index_1_etg_2_desc: tanh
|-activation_conv_index_1_etg_2_remont: relu
|-activation_conv_index_1_etg_3_desc: tanh
|-activation_conv_index_1_etg_3_remont: relu
|-activation_conv_index_1_etg_4_desc: relu
|-activation_conv_index_1_etg_4_remont: tanh
|-activation_conv_index_1_etg_5_remont: tanh
|-activation_conv_index_1_etg_6_desc: elu
|-activation_conv_index_1_etg_7_desc: selu
|-activation_conv_index_1_etg_7_remont: relu
|-activation_conv_index_1_etg_8_desc: relu
|-activation_conv_index_2_etg_0_desc: tanh
|-activation_conv_index_2_etg_0_remont: elu
|-activation_conv_index_2_etg_1_desc: elu
|-activation_conv_index_2_etg_2_desc: linear
|-activation_conv_index_2_etg_2_remont: tanh
|-activation_conv_index_2_etg_3_desc: relu
|-activation_conv_index_2_etg_3_remont: selu
|-activation_conv_index_2_etg_4_desc: linear
|-activation_conv_index_2_etg_4_remont: tanh
|-activation_conv_index_2_etg_7_desc: elu
|-activation_conv_index_2_etg_7_remont: elu
|-activation_conv_index_2_etg_8_desc: tanh
|-activation_conv_index_3_etg_0_remont: linear
|-activation_conv_index_3_etg_2_desc: relu
|-activation_conv_index_3_etg_7_remont: tanh
|-beta_1: 0.9765124137619023
|-beta_2: 0.8646209488191324
|-filtre_conv_index_0_etg_0_desc: 100
|-filtre_conv_index_0_etg_0_remont: 10
|-filtre_conv_index_0_etg_1_desc: 1
|-filtre_conv_index_0_etg_1_remont: 500
|-filtre_conv_index_0_etg_2_desc: 3
|-filtre_conv_index_0_etg_2_remont: 500
|-filtre_conv_index_0_etg_3_desc: 500
|-filtre_conv_index_0_etg_3_remont: 50
|-filtre_conv_index_0_etg_4_desc: 10
|-filtre_conv_index_0_etg_4_remont: 50
|-filtre_conv_index_0_etg_5_desc: 10
|-filtre_conv_index_0_etg_5_remont: 1
|-filtre_conv_index_0_etg_6_desc: 500
|-filtre_conv_index_0_etg_6_remont: 100
|-filtre_conv_index_0_etg_7_desc: 10
|-filtre_conv_index_0_etg_7_remont: 1
|-filtre_conv_index_0_etg_8_desc: 1
|-filtre_conv_index_1_etg_0_desc: 3
|-filtre_conv_index_1_etg_0_remont: 50
|-filtre_conv_index_1_etg_1_desc: 500
|-filtre_conv_index_1_etg_1_remont: 500
|-filtre_conv_index_1_etg_2_desc: 500
|-filtre_conv_index_1_etg_2_remont: 100
|-filtre_conv_index_1_etg_3_desc: 10
|-filtre_conv_index_1_etg_3_remont: 500
|-filtre_conv_index_1_etg_4_desc: 500
|-filtre_conv_index_1_etg_4_remont: 500
|-filtre_conv_index_1_etg_5_remont: 10
|-filtre_conv_index_1_etg_6_desc: 10
|-filtre_conv_index_1_etg_7_desc: 1
|-filtre_conv_index_1_etg_7_remont: 10
|-filtre_conv_index_1_etg_8_desc: 10
|-filtre_conv_index_2_etg_0_desc: 500
|-filtre_conv_index_2_etg_0_remont: 100
|-filtre_conv_index_2_etg_1_desc: 500
|-filtre_conv_index_2_etg_2_desc: 500
|-filtre_conv_index_2_etg_2_remont: 500
|-filtre_conv_index_2_etg_3_desc: 10
|-filtre_conv_index_2_etg_3_remont: 10
|-filtre_conv_index_2_etg_4_desc: 50
|-filtre_conv_index_2_etg_4_remont: 50
|-filtre_conv_index_2_etg_7_desc: 100
|-filtre_conv_index_2_etg_7_remont: 100
|-filtre_conv_index_2_etg_8_desc: 3
|-filtre_conv_index_3_etg_0_remont: 3
|-filtre_conv_index_3_etg_2_desc: 10
|-filtre_conv_index_3_etg_7_remont: 3
|-filtre_deconv_etg_0_remont: 3
|-filtre_deconv_etg_1_remont: 50
|-filtre_deconv_etg_2_remont: 10
|-filtre_deconv_etg_3_remont: 1
|-filtre_deconv_etg_4_remont: 500
|-filtre_deconv_etg_5_remont: 3
|-filtre_deconv_etg_6_remont: 500
|-filtre_deconv_etg_7_remont: 100
|-kernel_conv_index_0_etg_0_desc: 3
|-kernel_conv_index_0_etg_0_remont: 2
|-kernel_conv_index_0_etg_1_desc: 2
|-kernel_conv_index_0_etg_1_remont: 2
|-kernel_conv_index_0_etg_2_desc: 2
|-kernel_conv_index_0_etg_2_remont: 3
|-kernel_conv_index_0_etg_3_desc: 3
|-kernel_conv_index_0_etg_3_remont: 3
|-kernel_conv_index_0_etg_4_desc: 2
|-kernel_conv_index_0_etg_4_remont: 3
|-kernel_conv_index_0_etg_5_desc: 3
|-kernel_conv_index_0_etg_5_remont: 3
|-kernel_conv_index_0_etg_6_desc: 3
|-kernel_conv_index_0_etg_6_remont: 2
|-kernel_conv_index_0_etg_7_desc: 3
|-kernel_conv_index_0_etg_7_remont: 3
|-kernel_conv_index_0_etg_8_desc: 2
|-kernel_conv_index_1_etg_0_desc: 2
|-kernel_conv_index_1_etg_0_remont: 3
|-kernel_conv_index_1_etg_1_desc: 3
|-kernel_conv_index_1_etg_1_remont: 3
|-kernel_conv_index_1_etg_2_desc: 3
|-kernel_conv_index_1_etg_2_remont: 3
|-kernel_conv_index_1_etg_3_desc: 3
|-kernel_conv_index_1_etg_3_remont: 2
|-kernel_conv_index_1_etg_4_desc: 2
|-kernel_conv_index_1_etg_4_remont: 2
|-kernel_conv_index_1_etg_5_remont: 3
|-kernel_conv_index_1_etg_6_desc: 3
|-kernel_conv_index_1_etg_7_desc: 2
|-kernel_conv_index_1_etg_7_remont: 2
|-kernel_conv_index_1_etg_8_desc: 2
|-kernel_conv_index_2_etg_0_desc: 3
|-kernel_conv_index_2_etg_0_remont: 3
|-kernel_conv_index_2_etg_1_desc: 3
|-kernel_conv_index_2_etg_2_desc: 2
|-kernel_conv_index_2_etg_2_remont: 2
|-kernel_conv_index_2_etg_3_desc: 3
|-kernel_conv_index_2_etg_3_remont: 3
|-kernel_conv_index_2_etg_4_desc: 3
|-kernel_conv_index_2_etg_4_remont: 2
|-kernel_conv_index_2_etg_7_desc: 3
|-kernel_conv_index_2_etg_7_remont: 2
|-kernel_conv_index_2_etg_8_desc: 3
|-kernel_conv_index_3_etg_0_remont: 2
|-kernel_conv_index_3_etg_2_desc: 2
|-kernel_conv_index_3_etg_7_remont: 2
|-learning_rate: 0.001
|-nb_conv_etg_0_desc: 1
|-nb_conv_etg_0_remont: 3
|-nb_conv_etg_1_desc: 2
|-nb_conv_etg_1_remont: 2
|-nb_conv_etg_2_desc: 4
|-nb_conv_etg_2_remont: 1
|-nb_conv_etg_3_desc: 1
|-nb_conv_etg_3_remont: 1
|-nb_conv_etg_4_desc: 2
|-nb_conv_etg_4_remont: 2
|-nb_conv_etg_5_desc: 0
|-nb_conv_etg_5_remont: 2
|-nb_conv_etg_6_desc: 4
|-nb_conv_etg_6_remont: 1
|-nb_conv_etg_7_desc: 1
|-nb_conv_etg_7_remont: 0
|-nb_conv_etg_8_desc: 4
|-nb_pool: 0
|-pool_type_0: avg
|-pool_type_1: avg
|-pool_type_2: max
|-pool_type_3: avg
|-pool_type_4: max
|-pool_type_5: avg
|-pool_type_6: max
|-pool_type_7: avg
Trial summary
|-Trial ID: 2eef72f23e24f4a83aa8023a3b3121eb
|-Score: 0.5867435956994692
|-Best step: 0
Hyperparameters:
|-activation_conv_index_0_etg_0_desc: elu
|-activation_conv_index_0_etg_0_remont: relu
|-activation_conv_index_0_etg_1_desc: selu
|-activation_conv_index_0_etg_1_remont: selu
|-activation_conv_index_0_etg_2_desc: tanh
|-activation_conv_index_0_etg_2_remont: tanh
|-activation_conv_index_0_etg_3_desc: tanh
|-activation_conv_index_0_etg_3_remont: relu
|-activation_conv_index_0_etg_4_desc: linear
|-activation_conv_index_1_etg_0_desc: selu
|-activation_conv_index_1_etg_0_remont: relu
|-activation_conv_index_1_etg_1_desc: relu
|-activation_conv_index_1_etg_2_remont: relu
|-activation_conv_index_2_etg_0_desc: linear
|-activation_conv_index_2_etg_0_remont: relu
|-activation_conv_index_2_etg_1_desc: relu
|-activation_conv_index_2_etg_2_remont: relu
|-activation_conv_index_3_etg_0_remont: relu
|-beta_1: 1.064355438665868
|-beta_2: 0.8195811574388034
|-filtre_conv_index_0_etg_0_desc: 1
|-filtre_conv_index_0_etg_0_remont: 10
|-filtre_conv_index_0_etg_1_desc: 3
|-filtre_conv_index_0_etg_1_remont: 10
|-filtre_conv_index_0_etg_2_desc: 10
|-filtre_conv_index_0_etg_2_remont: 500
|-filtre_conv_index_0_etg_3_desc: 500
|-filtre_conv_index_0_etg_3_remont: 3
|-filtre_conv_index_0_etg_4_desc: 500
|-filtre_conv_index_1_etg_0_desc: 50
|-filtre_conv_index_1_etg_0_remont: 1
|-filtre_conv_index_1_etg_1_desc: 1
|-filtre_conv_index_1_etg_2_remont: 1
|-filtre_conv_index_2_etg_0_desc: 3
|-filtre_conv_index_2_etg_0_remont: 1
|-filtre_conv_index_2_etg_1_desc: 1
|-filtre_conv_index_2_etg_2_remont: 1
|-filtre_conv_index_3_etg_0_remont: 1
|-filtre_deconv_etg_0_remont: 10
|-filtre_deconv_etg_1_remont: 100
|-filtre_deconv_etg_2_remont: 1
|-filtre_deconv_etg_3_remont: 1
|-kernel_conv_index_0_etg_0_desc: 3
|-kernel_conv_index_0_etg_0_remont: 2
|-kernel_conv_index_0_etg_1_desc: 2
|-kernel_conv_index_0_etg_1_remont: 3
|-kernel_conv_index_0_etg_2_desc: 3
|-kernel_conv_index_0_etg_2_remont: 3
|-kernel_conv_index_0_etg_3_desc: 3
|-kernel_conv_index_0_etg_3_remont: 3
|-kernel_conv_index_0_etg_4_desc: 2
|-kernel_conv_index_1_etg_0_desc: 2
|-kernel_conv_index_1_etg_0_remont: 2
|-kernel_conv_index_1_etg_1_desc: 2
|-kernel_conv_index_1_etg_2_remont: 2
|-kernel_conv_index_2_etg_0_desc: 2
|-kernel_conv_index_2_etg_0_remont: 2
|-kernel_conv_index_2_etg_1_desc: 2
|-kernel_conv_index_2_etg_2_remont: 2
|-kernel_conv_index_3_etg_0_remont: 2
|-learning_rate: 1e-05
|-nb_conv_etg_0_desc: 1
|-nb_conv_etg_0_remont: 4
|-nb_conv_etg_1_desc: 3
|-nb_conv_etg_1_remont: 1
|-nb_conv_etg_2_desc: 0
|-nb_conv_etg_2_remont: 3
|-nb_conv_etg_3_desc: 1
|-nb_conv_etg_3_remont: 1
|-nb_conv_etg_4_desc: 2
|-nb_pool: 3
|-pool_type_0: max
|-pool_type_1: avg
|-pool_type_2: avg
|-pool_type_3: avg
Trial summary
|-Trial ID: 9aea87148abf20fdbb03daf7564eb9a7
|-Score: 0.34692077338695526
|-Best step: 0
Hyperparameters:
|-activation_conv_index_0_etg_0_desc: elu
|-activation_conv_index_0_etg_0_remont: tanh
|-activation_conv_index_0_etg_1_desc: tanh
|-activation_conv_index_0_etg_1_remont: tanh
|-activation_conv_index_0_etg_2_desc: relu
|-activation_conv_index_0_etg_2_remont: linear
|-activation_conv_index_0_etg_3_desc: tanh
|-activation_conv_index_0_etg_3_remont: tanh
|-activation_conv_index_0_etg_4_desc: tanh
|-activation_conv_index_0_etg_4_remont: selu
|-activation_conv_index_0_etg_5_desc: elu
|-activation_conv_index_0_etg_5_remont: elu
|-activation_conv_index_0_etg_6_desc: relu
|-activation_conv_index_0_etg_6_remont: tanh
|-activation_conv_index_0_etg_7_desc: elu
|-activation_conv_index_0_etg_7_remont: relu
|-activation_conv_index_0_etg_8_desc: relu
|-activation_conv_index_1_etg_0_desc: relu
|-activation_conv_index_1_etg_0_remont: tanh
|-activation_conv_index_1_etg_1_desc: elu
|-activation_conv_index_1_etg_1_remont: selu
|-activation_conv_index_1_etg_2_desc: selu
|-activation_conv_index_1_etg_2_remont: linear
|-activation_conv_index_1_etg_3_desc: relu
|-activation_conv_index_1_etg_3_remont: elu
|-activation_conv_index_1_etg_4_desc: relu
|-activation_conv_index_1_etg_4_remont: relu
|-activation_conv_index_1_etg_5_desc: selu
|-activation_conv_index_1_etg_5_remont: relu
|-activation_conv_index_1_etg_6_desc: relu
|-activation_conv_index_1_etg_6_remont: elu
|-activation_conv_index_1_etg_7_desc: selu
|-activation_conv_index_1_etg_7_remont: tanh
|-activation_conv_index_1_etg_8_desc: relu
|-activation_conv_index_2_etg_0_desc: tanh
|-activation_conv_index_2_etg_0_remont: relu
|-activation_conv_index_2_etg_1_desc: relu
|-activation_conv_index_2_etg_1_remont: tanh
|-activation_conv_index_2_etg_2_desc: linear
|-activation_conv_index_2_etg_2_remont: tanh
|-activation_conv_index_2_etg_3_desc: linear
|-activation_conv_index_2_etg_3_remont: tanh
|-activation_conv_index_2_etg_4_desc: relu
|-activation_conv_index_2_etg_4_remont: tanh
|-activation_conv_index_2_etg_5_remont: linear
|-activation_conv_index_2_etg_7_desc: linear
|-activation_conv_index_2_etg_7_remont: selu
|-activation_conv_index_2_etg_8_desc: linear
|-activation_conv_index_3_etg_0_desc: relu
|-activation_conv_index_3_etg_0_remont: selu
|-activation_conv_index_3_etg_1_desc: selu
|-activation_conv_index_3_etg_1_remont: tanh
|-activation_conv_index_3_etg_2_desc: selu
|-activation_conv_index_3_etg_4_desc: selu
|-activation_conv_index_3_etg_7_remont: tanh
|-beta_1: 0.9185960187359232
|-beta_2: 0.9542589124228112
|-filtre_conv_index_0_etg_0_desc: 500
|-filtre_conv_index_0_etg_0_remont: 10
|-filtre_conv_index_0_etg_1_desc: 10
|-filtre_conv_index_0_etg_1_remont: 3
|-filtre_conv_index_0_etg_2_desc: 1
|-filtre_conv_index_0_etg_2_remont: 500
|-filtre_conv_index_0_etg_3_desc: 100
|-filtre_conv_index_0_etg_3_remont: 50
|-filtre_conv_index_0_etg_4_desc: 500
|-filtre_conv_index_0_etg_4_remont: 10
|-filtre_conv_index_0_etg_5_desc: 10
|-filtre_conv_index_0_etg_5_remont: 1
|-filtre_conv_index_0_etg_6_desc: 500
|-filtre_conv_index_0_etg_6_remont: 10
|-filtre_conv_index_0_etg_7_desc: 3
|-filtre_conv_index_0_etg_7_remont: 100
|-filtre_conv_index_0_etg_8_desc: 100
|-filtre_conv_index_1_etg_0_desc: 1
|-filtre_conv_index_1_etg_0_remont: 10
|-filtre_conv_index_1_etg_1_desc: 50
|-filtre_conv_index_1_etg_1_remont: 500
|-filtre_conv_index_1_etg_2_desc: 500
|-filtre_conv_index_1_etg_2_remont: 100
|-filtre_conv_index_1_etg_3_desc: 10
|-filtre_conv_index_1_etg_3_remont: 500
|-filtre_conv_index_1_etg_4_desc: 500
|-filtre_conv_index_1_etg_4_remont: 500
|-filtre_conv_index_1_etg_5_desc: 100
|-filtre_conv_index_1_etg_5_remont: 100
|-filtre_conv_index_1_etg_6_desc: 100
|-filtre_conv_index_1_etg_6_remont: 1
|-filtre_conv_index_1_etg_7_desc: 1
|-filtre_conv_index_1_etg_7_remont: 100
|-filtre_conv_index_1_etg_8_desc: 1
|-filtre_conv_index_2_etg_0_desc: 50
|-filtre_conv_index_2_etg_0_remont: 50
|-filtre_conv_index_2_etg_1_desc: 50
|-filtre_conv_index_2_etg_1_remont: 500
|-filtre_conv_index_2_etg_2_desc: 500
|-filtre_conv_index_2_etg_2_remont: 50
|-filtre_conv_index_2_etg_3_desc: 100
|-filtre_conv_index_2_etg_3_remont: 500
|-filtre_conv_index_2_etg_4_desc: 100
|-filtre_conv_index_2_etg_4_remont: 3
|-filtre_conv_index_2_etg_5_remont: 500
|-filtre_conv_index_2_etg_7_desc: 100
|-filtre_conv_index_2_etg_7_remont: 50
|-filtre_conv_index_2_etg_8_desc: 500
|-filtre_conv_index_3_etg_0_desc: 1
|-filtre_conv_index_3_etg_0_remont: 3
|-filtre_conv_index_3_etg_1_desc: 50
|-filtre_conv_index_3_etg_1_remont: 10
|-filtre_conv_index_3_etg_2_desc: 3
|-filtre_conv_index_3_etg_4_desc: 10
|-filtre_conv_index_3_etg_7_remont: 500
|-filtre_deconv_etg_0_remont: 1
|-filtre_deconv_etg_1_remont: 10
|-filtre_deconv_etg_2_remont: 10
|-filtre_deconv_etg_3_remont: 50
|-filtre_deconv_etg_4_remont: 1
|-filtre_deconv_etg_5_remont: 1
|-filtre_deconv_etg_6_remont: 1
|-filtre_deconv_etg_7_remont: 100
|-kernel_conv_index_0_etg_0_desc: 2
|-kernel_conv_index_0_etg_0_remont: 2
|-kernel_conv_index_0_etg_1_desc: 2
|-kernel_conv_index_0_etg_1_remont: 2
|-kernel_conv_index_0_etg_2_desc: 2
|-kernel_conv_index_0_etg_2_remont: 2
|-kernel_conv_index_0_etg_3_desc: 2
|-kernel_conv_index_0_etg_3_remont: 2
|-kernel_conv_index_0_etg_4_desc: 3
|-kernel_conv_index_0_etg_4_remont: 2
|-kernel_conv_index_0_etg_5_desc: 3
|-kernel_conv_index_0_etg_5_remont: 2
|-kernel_conv_index_0_etg_6_desc: 2
|-kernel_conv_index_0_etg_6_remont: 2
|-kernel_conv_index_0_etg_7_desc: 2
|-kernel_conv_index_0_etg_7_remont: 3
|-kernel_conv_index_0_etg_8_desc: 2
|-kernel_conv_index_1_etg_0_desc: 3
|-kernel_conv_index_1_etg_0_remont: 3
|-kernel_conv_index_1_etg_1_desc: 3
|-kernel_conv_index_1_etg_1_remont: 3
|-kernel_conv_index_1_etg_2_desc: 2
|-kernel_conv_index_1_etg_2_remont: 2
|-kernel_conv_index_1_etg_3_desc: 2
|-kernel_conv_index_1_etg_3_remont: 2
|-kernel_conv_index_1_etg_4_desc: 3
|-kernel_conv_index_1_etg_4_remont: 2
|-kernel_conv_index_1_etg_5_desc: 3
|-kernel_conv_index_1_etg_5_remont: 3
|-kernel_conv_index_1_etg_6_desc: 3
|-kernel_conv_index_1_etg_6_remont: 3
|-kernel_conv_index_1_etg_7_desc: 2
|-kernel_conv_index_1_etg_7_remont: 2
|-kernel_conv_index_1_etg_8_desc: 2
|-kernel_conv_index_2_etg_0_desc: 2
|-kernel_conv_index_2_etg_0_remont: 2
|-kernel_conv_index_2_etg_1_desc: 3
|-kernel_conv_index_2_etg_1_remont: 3
|-kernel_conv_index_2_etg_2_desc: 3
|-kernel_conv_index_2_etg_2_remont: 2
|-kernel_conv_index_2_etg_3_desc: 3
|-kernel_conv_index_2_etg_3_remont: 3
|-kernel_conv_index_2_etg_4_desc: 2
|-kernel_conv_index_2_etg_4_remont: 2
|-kernel_conv_index_2_etg_5_remont: 2
|-kernel_conv_index_2_etg_7_desc: 2
|-kernel_conv_index_2_etg_7_remont: 3
|-kernel_conv_index_2_etg_8_desc: 2
|-kernel_conv_index_3_etg_0_desc: 2
|-kernel_conv_index_3_etg_0_remont: 2
|-kernel_conv_index_3_etg_1_desc: 2
|-kernel_conv_index_3_etg_1_remont: 3
|-kernel_conv_index_3_etg_2_desc: 3
|-kernel_conv_index_3_etg_4_desc: 3
|-kernel_conv_index_3_etg_7_remont: 2
|-learning_rate: 1e-05
|-nb_conv_etg_0_desc: 4
|-nb_conv_etg_0_remont: 1
|-nb_conv_etg_1_desc: 3
|-nb_conv_etg_1_remont: 3
|-nb_conv_etg_2_desc: 3
|-nb_conv_etg_2_remont: 0
|-nb_conv_etg_3_desc: 3
|-nb_conv_etg_3_remont: 2
|-nb_conv_etg_4_desc: 4
|-nb_conv_etg_4_remont: 4
|-nb_conv_etg_5_desc: 1
|-nb_conv_etg_5_remont: 2
|-nb_conv_etg_6_desc: 3
|-nb_conv_etg_6_remont: 1
|-nb_conv_etg_7_desc: 4
|-nb_conv_etg_7_remont: 2
|-nb_conv_etg_8_desc: 3
|-nb_pool: 0
|-pool_type_0: avg
|-pool_type_1: max
|-pool_type_2: max
|-pool_type_3: avg
|-pool_type_4: max
|-pool_type_5: max
|-pool_type_6: avg
|-pool_type_7: avg
Trial summary
|-Trial ID: b9b474249eae2dcf5e368004c8d66e1a
|-Score: 0.26092660675446194
|-Best step: 0
Hyperparameters:
|-activation_conv_index_0_etg_0_desc: tanh
|-activation_conv_index_0_etg_0_remont: tanh
|-activation_conv_index_0_etg_1_desc: tanh
|-activation_conv_index_0_etg_1_remont: relu
|-activation_conv_index_0_etg_2_desc: elu
|-activation_conv_index_0_etg_2_remont: tanh
|-activation_conv_index_0_etg_3_desc: selu
|-activation_conv_index_0_etg_3_remont: linear
|-activation_conv_index_0_etg_4_desc: elu
|-activation_conv_index_0_etg_4_remont: relu
|-activation_conv_index_0_etg_5_desc: tanh
|-activation_conv_index_0_etg_5_remont: selu
|-activation_conv_index_0_etg_6_desc: tanh
|-activation_conv_index_0_etg_6_remont: tanh
|-activation_conv_index_0_etg_7_desc: elu
|-activation_conv_index_0_etg_7_remont: tanh
|-activation_conv_index_0_etg_8_desc: elu
|-activation_conv_index_1_etg_0_desc: tanh
|-activation_conv_index_1_etg_0_remont: selu
|-activation_conv_index_1_etg_1_desc: linear
|-activation_conv_index_1_etg_1_remont: linear
|-activation_conv_index_1_etg_2_desc: relu
|-activation_conv_index_1_etg_2_remont: tanh
|-activation_conv_index_1_etg_3_desc: tanh
|-activation_conv_index_1_etg_3_remont: tanh
|-activation_conv_index_1_etg_4_desc: elu
|-activation_conv_index_1_etg_4_remont: relu
|-activation_conv_index_1_etg_5_remont: relu
|-activation_conv_index_1_etg_6_desc: selu
|-activation_conv_index_1_etg_6_remont: relu
|-activation_conv_index_1_etg_7_desc: elu
|-activation_conv_index_1_etg_7_remont: linear
|-activation_conv_index_1_etg_8_desc: linear
|-activation_conv_index_2_etg_0_desc: elu
|-activation_conv_index_2_etg_0_remont: selu
|-activation_conv_index_2_etg_1_desc: elu
|-activation_conv_index_2_etg_1_remont: relu
|-activation_conv_index_2_etg_2_desc: tanh
|-activation_conv_index_2_etg_2_remont: elu
|-activation_conv_index_2_etg_3_desc: linear
|-activation_conv_index_2_etg_3_remont: elu
|-activation_conv_index_2_etg_4_desc: relu
|-activation_conv_index_2_etg_4_remont: linear
|-activation_conv_index_2_etg_5_remont: relu
|-activation_conv_index_2_etg_7_desc: linear
|-activation_conv_index_2_etg_7_remont: tanh
|-activation_conv_index_2_etg_8_desc: tanh
|-activation_conv_index_3_etg_0_remont: selu
|-activation_conv_index_3_etg_1_remont: relu
|-activation_conv_index_3_etg_2_desc: linear
|-activation_conv_index_3_etg_4_desc: relu
|-activation_conv_index_3_etg_7_remont: selu
|-beta_1: 1.1675300436610982
|-beta_2: 0.9222156939959956
|-filtre_conv_index_0_etg_0_desc: 10
|-filtre_conv_index_0_etg_0_remont: 3
|-filtre_conv_index_0_etg_1_desc: 1
|-filtre_conv_index_0_etg_1_remont: 100
|-filtre_conv_index_0_etg_2_desc: 500
|-filtre_conv_index_0_etg_2_remont: 100
|-filtre_conv_index_0_etg_3_desc: 10
|-filtre_conv_index_0_etg_3_remont: 500
|-filtre_conv_index_0_etg_4_desc: 10
|-filtre_conv_index_0_etg_4_remont: 50
|-filtre_conv_index_0_etg_5_desc: 10
|-filtre_conv_index_0_etg_5_remont: 1
|-filtre_conv_index_0_etg_6_desc: 100
|-filtre_conv_index_0_etg_6_remont: 100
|-filtre_conv_index_0_etg_7_desc: 500
|-filtre_conv_index_0_etg_7_remont: 3
|-filtre_conv_index_0_etg_8_desc: 50
|-filtre_conv_index_1_etg_0_desc: 10
|-filtre_conv_index_1_etg_0_remont: 10
|-filtre_conv_index_1_etg_1_desc: 10
|-filtre_conv_index_1_etg_1_remont: 3
|-filtre_conv_index_1_etg_2_desc: 1
|-filtre_conv_index_1_etg_2_remont: 100
|-filtre_conv_index_1_etg_3_desc: 500
|-filtre_conv_index_1_etg_3_remont: 10
|-filtre_conv_index_1_etg_4_desc: 10
|-filtre_conv_index_1_etg_4_remont: 10
|-filtre_conv_index_1_etg_5_remont: 500
|-filtre_conv_index_1_etg_6_desc: 100
|-filtre_conv_index_1_etg_6_remont: 1
|-filtre_conv_index_1_etg_7_desc: 100
|-filtre_conv_index_1_etg_7_remont: 500
|-filtre_conv_index_1_etg_8_desc: 100
|-filtre_conv_index_2_etg_0_desc: 1
|-filtre_conv_index_2_etg_0_remont: 100
|-filtre_conv_index_2_etg_1_desc: 1
|-filtre_conv_index_2_etg_1_remont: 1
|-filtre_conv_index_2_etg_2_desc: 10
|-filtre_conv_index_2_etg_2_remont: 50
|-filtre_conv_index_2_etg_3_desc: 10
|-filtre_conv_index_2_etg_3_remont: 3
|-filtre_conv_index_2_etg_4_desc: 50
|-filtre_conv_index_2_etg_4_remont: 100
|-filtre_conv_index_2_etg_5_remont: 1
|-filtre_conv_index_2_etg_7_desc: 100
|-filtre_conv_index_2_etg_7_remont: 3
|-filtre_conv_index_2_etg_8_desc: 500
|-filtre_conv_index_3_etg_0_remont: 100
|-filtre_conv_index_3_etg_1_remont: 1
|-filtre_conv_index_3_etg_2_desc: 1
|-filtre_conv_index_3_etg_4_desc: 1
|-filtre_conv_index_3_etg_7_remont: 3
|-filtre_deconv_etg_0_remont: 50
|-filtre_deconv_etg_1_remont: 3
|-filtre_deconv_etg_2_remont: 1
|-filtre_deconv_etg_3_remont: 100
|-filtre_deconv_etg_4_remont: 3
|-filtre_deconv_etg_5_remont: 100
|-filtre_deconv_etg_6_remont: 500
|-filtre_deconv_etg_7_remont: 3
|-kernel_conv_index_0_etg_0_desc: 2
|-kernel_conv_index_0_etg_0_remont: 2
|-kernel_conv_index_0_etg_1_desc: 3
|-kernel_conv_index_0_etg_1_remont: 3
|-kernel_conv_index_0_etg_2_desc: 3
|-kernel_conv_index_0_etg_2_remont: 2
|-kernel_conv_index_0_etg_3_desc: 3
|-kernel_conv_index_0_etg_3_remont: 3
|-kernel_conv_index_0_etg_4_desc: 2
|-kernel_conv_index_0_etg_4_remont: 2
|-kernel_conv_index_0_etg_5_desc: 3
|-kernel_conv_index_0_etg_5_remont: 3
|-kernel_conv_index_0_etg_6_desc: 3
|-kernel_conv_index_0_etg_6_remont: 2
|-kernel_conv_index_0_etg_7_desc: 2
|-kernel_conv_index_0_etg_7_remont: 2
|-kernel_conv_index_0_etg_8_desc: 3
|-kernel_conv_index_1_etg_0_desc: 3
|-kernel_conv_index_1_etg_0_remont: 2
|-kernel_conv_index_1_etg_1_desc: 3
|-kernel_conv_index_1_etg_1_remont: 3
|-kernel_conv_index_1_etg_2_desc: 2
|-kernel_conv_index_1_etg_2_remont: 2
|-kernel_conv_index_1_etg_3_desc: 2
|-kernel_conv_index_1_etg_3_remont: 2
|-kernel_conv_index_1_etg_4_desc: 3
|-kernel_conv_index_1_etg_4_remont: 3
|-kernel_conv_index_1_etg_5_remont: 2
|-kernel_conv_index_1_etg_6_desc: 2
|-kernel_conv_index_1_etg_6_remont: 2
|-kernel_conv_index_1_etg_7_desc: 3
|-kernel_conv_index_1_etg_7_remont: 2
|-kernel_conv_index_1_etg_8_desc: 3
|-kernel_conv_index_2_etg_0_desc: 2
|-kernel_conv_index_2_etg_0_remont: 3
|-kernel_conv_index_2_etg_1_desc: 3
|-kernel_conv_index_2_etg_1_remont: 2
|-kernel_conv_index_2_etg_2_desc: 2
|-kernel_conv_index_2_etg_2_remont: 3
|-kernel_conv_index_2_etg_3_desc: 3
|-kernel_conv_index_2_etg_3_remont: 2
|-kernel_conv_index_2_etg_4_desc: 2
|-kernel_conv_index_2_etg_4_remont: 3
|-kernel_conv_index_2_etg_5_remont: 2
|-kernel_conv_index_2_etg_7_desc: 2
|-kernel_conv_index_2_etg_7_remont: 3
|-kernel_conv_index_2_etg_8_desc: 3
|-kernel_conv_index_3_etg_0_remont: 2
|-kernel_conv_index_3_etg_1_remont: 2
|-kernel_conv_index_3_etg_2_desc: 3
|-kernel_conv_index_3_etg_4_desc: 2
|-kernel_conv_index_3_etg_7_remont: 2
|-learning_rate: 0.0001
|-nb_conv_etg_0_desc: 1
|-nb_conv_etg_0_remont: 1
|-nb_conv_etg_1_desc: 2
|-nb_conv_etg_1_remont: 4
|-nb_conv_etg_2_desc: 1
|-nb_conv_etg_2_remont: 2
|-nb_conv_etg_3_desc: 1
|-nb_conv_etg_3_remont: 3
|-nb_conv_etg_4_desc: 4
|-nb_conv_etg_4_remont: 1
|-nb_conv_etg_5_desc: 0
|-nb_conv_etg_5_remont: 3
|-nb_conv_etg_6_desc: 1
|-nb_conv_etg_6_remont: 2
|-nb_conv_etg_7_desc: 2
|-nb_conv_etg_7_remont: 0
|-nb_conv_etg_8_desc: 3
|-nb_pool: 8
|-pool_type_0: avg
|-pool_type_1: max
|-pool_type_2: avg
|-pool_type_3: max
|-pool_type_4: avg
|-pool_type_5: avg
|-pool_type_6: avg
|-pool_type_7: max
Trial summary
|-Trial ID: dbb80755bd84745f7d52cdc5f4764c8a
|-Score: 0.1504457692305247
|-Best step: 0
Hyperparameters:
|-activation_conv_index_0_etg_0_desc: relu
|-activation_conv_index_0_etg_0_remont: tanh
|-activation_conv_index_0_etg_1_desc: relu
|-activation_conv_index_0_etg_1_remont: tanh
|-activation_conv_index_0_etg_2_desc: relu
|-activation_conv_index_0_etg_2_remont: tanh
|-activation_conv_index_0_etg_3_desc: elu
|-activation_conv_index_0_etg_3_remont: tanh
|-activation_conv_index_0_etg_4_desc: selu
|-activation_conv_index_0_etg_4_remont: selu
|-activation_conv_index_0_etg_5_desc: linear
|-activation_conv_index_0_etg_5_remont: elu
|-activation_conv_index_0_etg_6_desc: linear
|-activation_conv_index_0_etg_6_remont: linear
|-activation_conv_index_0_etg_7_desc: elu
|-activation_conv_index_0_etg_7_remont: elu
|-activation_conv_index_0_etg_8_desc: elu
|-activation_conv_index_1_etg_0_desc: relu
|-activation_conv_index_1_etg_0_remont: selu
|-activation_conv_index_1_etg_1_desc: relu
|-activation_conv_index_1_etg_1_remont: elu
|-activation_conv_index_1_etg_2_desc: linear
|-activation_conv_index_1_etg_2_remont: linear
|-activation_conv_index_1_etg_3_desc: tanh
|-activation_conv_index_1_etg_3_remont: elu
|-activation_conv_index_1_etg_4_desc: selu
|-activation_conv_index_1_etg_4_remont: elu
|-activation_conv_index_1_etg_5_desc: relu
|-activation_conv_index_1_etg_5_remont: tanh
|-activation_conv_index_1_etg_6_desc: relu
|-activation_conv_index_1_etg_6_remont: linear
|-activation_conv_index_1_etg_7_desc: relu
|-activation_conv_index_1_etg_7_remont: selu
|-activation_conv_index_1_etg_8_desc: linear
|-activation_conv_index_2_etg_0_desc: linear
|-activation_conv_index_2_etg_0_remont: relu
|-activation_conv_index_2_etg_1_desc: tanh
|-activation_conv_index_2_etg_1_remont: selu
|-activation_conv_index_2_etg_2_desc: tanh
|-activation_conv_index_2_etg_2_remont: linear
|-activation_conv_index_2_etg_3_desc: elu
|-activation_conv_index_2_etg_3_remont: linear
|-activation_conv_index_2_etg_4_desc: relu
|-activation_conv_index_2_etg_4_remont: elu
|-activation_conv_index_2_etg_5_remont: tanh
|-activation_conv_index_2_etg_7_desc: relu
|-activation_conv_index_2_etg_7_remont: elu
|-activation_conv_index_2_etg_8_desc: linear
|-activation_conv_index_3_etg_0_remont: linear
|-activation_conv_index_3_etg_1_desc: relu
|-activation_conv_index_3_etg_1_remont: relu
|-activation_conv_index_3_etg_2_desc: selu
|-activation_conv_index_3_etg_4_desc: elu
|-activation_conv_index_3_etg_7_remont: linear
|-beta_1: 1.0290648554975916
|-beta_2: 0.8884161545749057
|-filtre_conv_index_0_etg_0_desc: 3
|-filtre_conv_index_0_etg_0_remont: 500
|-filtre_conv_index_0_etg_1_desc: 10
|-filtre_conv_index_0_etg_1_remont: 3
|-filtre_conv_index_0_etg_2_desc: 1
|-filtre_conv_index_0_etg_2_remont: 10
|-filtre_conv_index_0_etg_3_desc: 10
|-filtre_conv_index_0_etg_3_remont: 1
|-filtre_conv_index_0_etg_4_desc: 3
|-filtre_conv_index_0_etg_4_remont: 10
|-filtre_conv_index_0_etg_5_desc: 50
|-filtre_conv_index_0_etg_5_remont: 10
|-filtre_conv_index_0_etg_6_desc: 50
|-filtre_conv_index_0_etg_6_remont: 1
|-filtre_conv_index_0_etg_7_desc: 3
|-filtre_conv_index_0_etg_7_remont: 3
|-filtre_conv_index_0_etg_8_desc: 10
|-filtre_conv_index_1_etg_0_desc: 500
|-filtre_conv_index_1_etg_0_remont: 10
|-filtre_conv_index_1_etg_1_desc: 10
|-filtre_conv_index_1_etg_1_remont: 500
|-filtre_conv_index_1_etg_2_desc: 3
|-filtre_conv_index_1_etg_2_remont: 50
|-filtre_conv_index_1_etg_3_desc: 50
|-filtre_conv_index_1_etg_3_remont: 50
|-filtre_conv_index_1_etg_4_desc: 500
|-filtre_conv_index_1_etg_4_remont: 50
|-filtre_conv_index_1_etg_5_desc: 1
|-filtre_conv_index_1_etg_5_remont: 100
|-filtre_conv_index_1_etg_6_desc: 10
|-filtre_conv_index_1_etg_6_remont: 100
|-filtre_conv_index_1_etg_7_desc: 100
|-filtre_conv_index_1_etg_7_remont: 100
|-filtre_conv_index_1_etg_8_desc: 500
|-filtre_conv_index_2_etg_0_desc: 10
|-filtre_conv_index_2_etg_0_remont: 1
|-filtre_conv_index_2_etg_1_desc: 100
|-filtre_conv_index_2_etg_1_remont: 3
|-filtre_conv_index_2_etg_2_desc: 500
|-filtre_conv_index_2_etg_2_remont: 1
|-filtre_conv_index_2_etg_3_desc: 10
|-filtre_conv_index_2_etg_3_remont: 3
|-filtre_conv_index_2_etg_4_desc: 50
|-filtre_conv_index_2_etg_4_remont: 10
|-filtre_conv_index_2_etg_5_remont: 3
|-filtre_conv_index_2_etg_7_desc: 3
|-filtre_conv_index_2_etg_7_remont: 1
|-filtre_conv_index_2_etg_8_desc: 50
|-filtre_conv_index_3_etg_0_remont: 3
|-filtre_conv_index_3_etg_1_desc: 1
|-filtre_conv_index_3_etg_1_remont: 3
|-filtre_conv_index_3_etg_2_desc: 100
|-filtre_conv_index_3_etg_4_desc: 500
|-filtre_conv_index_3_etg_7_remont: 100
|-filtre_deconv_etg_0_remont: 1
|-filtre_deconv_etg_1_remont: 500
|-filtre_deconv_etg_2_remont: 1
|-filtre_deconv_etg_3_remont: 100
|-filtre_deconv_etg_4_remont: 10
|-filtre_deconv_etg_5_remont: 100
|-filtre_deconv_etg_6_remont: 1
|-filtre_deconv_etg_7_remont: 50
|-kernel_conv_index_0_etg_0_desc: 2
|-kernel_conv_index_0_etg_0_remont: 3
|-kernel_conv_index_0_etg_1_desc: 2
|-kernel_conv_index_0_etg_1_remont: 3
|-kernel_conv_index_0_etg_2_desc: 3
|-kernel_conv_index_0_etg_2_remont: 3
|-kernel_conv_index_0_etg_3_desc: 2
|-kernel_conv_index_0_etg_3_remont: 3
|-kernel_conv_index_0_etg_4_desc: 3
|-kernel_conv_index_0_etg_4_remont: 2
|-kernel_conv_index_0_etg_5_desc: 3
|-kernel_conv_index_0_etg_5_remont: 2
|-kernel_conv_index_0_etg_6_desc: 3
|-kernel_conv_index_0_etg_6_remont: 2
|-kernel_conv_index_0_etg_7_desc: 3
|-kernel_conv_index_0_etg_7_remont: 3
|-kernel_conv_index_0_etg_8_desc: 2
|-kernel_conv_index_1_etg_0_desc: 2
|-kernel_conv_index_1_etg_0_remont: 2
|-kernel_conv_index_1_etg_1_desc: 2
|-kernel_conv_index_1_etg_1_remont: 3
|-kernel_conv_index_1_etg_2_desc: 3
|-kernel_conv_index_1_etg_2_remont: 2
|-kernel_conv_index_1_etg_3_desc: 3
|-kernel_conv_index_1_etg_3_remont: 2
|-kernel_conv_index_1_etg_4_desc: 3
|-kernel_conv_index_1_etg_4_remont: 2
|-kernel_conv_index_1_etg_5_desc: 2
|-kernel_conv_index_1_etg_5_remont: 3
|-kernel_conv_index_1_etg_6_desc: 3
|-kernel_conv_index_1_etg_6_remont: 3
|-kernel_conv_index_1_etg_7_desc: 3
|-kernel_conv_index_1_etg_7_remont: 2
|-kernel_conv_index_1_etg_8_desc: 3
|-kernel_conv_index_2_etg_0_desc: 2
|-kernel_conv_index_2_etg_0_remont: 3
|-kernel_conv_index_2_etg_1_desc: 3
|-kernel_conv_index_2_etg_1_remont: 2
|-kernel_conv_index_2_etg_2_desc: 2
|-kernel_conv_index_2_etg_2_remont: 2
|-kernel_conv_index_2_etg_3_desc: 2
|-kernel_conv_index_2_etg_3_remont: 2
|-kernel_conv_index_2_etg_4_desc: 3
|-kernel_conv_index_2_etg_4_remont: 2
|-kernel_conv_index_2_etg_5_remont: 2
|-kernel_conv_index_2_etg_7_desc: 2
|-kernel_conv_index_2_etg_7_remont: 3
|-kernel_conv_index_2_etg_8_desc: 3
|-kernel_conv_index_3_etg_0_remont: 2
|-kernel_conv_index_3_etg_1_desc: 2
|-kernel_conv_index_3_etg_1_remont: 3
|-kernel_conv_index_3_etg_2_desc: 2
|-kernel_conv_index_3_etg_4_desc: 2
|-kernel_conv_index_3_etg_7_remont: 3
|-learning_rate: 1e-05
|-nb_conv_etg_0_desc: 2
|-nb_conv_etg_0_remont: 2
|-nb_conv_etg_1_desc: 4
|-nb_conv_etg_1_remont: 2
|-nb_conv_etg_2_desc: 4
|-nb_conv_etg_2_remont: 3
|-nb_conv_etg_3_desc: 3
|-nb_conv_etg_3_remont: 3
|-nb_conv_etg_4_desc: 1
|-nb_conv_etg_4_remont: 2
|-nb_conv_etg_5_desc: 2
|-nb_conv_etg_5_remont: 1
|-nb_conv_etg_6_desc: 0
|-nb_conv_etg_6_remont: 4
|-nb_conv_etg_7_desc: 3
|-nb_conv_etg_7_remont: 4
|-nb_conv_etg_8_desc: 4
|-nb_pool: 6
|-pool_type_0: avg
|-pool_type_1: max
|-pool_type_2: avg
|-pool_type_3: max
|-pool_type_4: max
|-pool_type_5: avg
|-pool_type_6: avg
|-pool_type_7: avg
Trial summary
|-Trial ID: 69b5b13a94692f068feb6f03bb7ffa7c
|-Score: 0.14004004001617432
|-Best step: 0
Hyperparameters:
|-activation_conv_index_0_etg_0_desc: selu
|-activation_conv_index_0_etg_0_remont: relu
|-activation_conv_index_0_etg_1_desc: relu
|-activation_conv_index_0_etg_1_remont: relu
|-activation_conv_index_0_etg_2_desc: relu
|-activation_conv_index_0_etg_2_remont: relu
|-activation_conv_index_0_etg_3_desc: relu
|-activation_conv_index_0_etg_3_remont: relu
|-activation_conv_index_0_etg_4_desc: relu
|-activation_conv_index_1_etg_0_desc: relu
|-activation_conv_index_2_etg_0_desc: relu
|-beta_1: 0.9840279767813848
|-beta_2: 0.961829610530099
|-filtre_conv_index_0_etg_0_desc: 50
|-filtre_conv_index_0_etg_0_remont: 1
|-filtre_conv_index_0_etg_1_desc: 1
|-filtre_conv_index_0_etg_1_remont: 1
|-filtre_conv_index_0_etg_2_desc: 1
|-filtre_conv_index_0_etg_2_remont: 1
|-filtre_conv_index_0_etg_3_desc: 1
|-filtre_conv_index_0_etg_3_remont: 1
|-filtre_conv_index_0_etg_4_desc: 1
|-filtre_conv_index_1_etg_0_desc: 1
|-filtre_conv_index_2_etg_0_desc: 1
|-filtre_deconv_etg_0_remont: 1
|-filtre_deconv_etg_1_remont: 1
|-filtre_deconv_etg_2_remont: 1
|-filtre_deconv_etg_3_remont: 1
|-kernel_conv_index_0_etg_0_desc: 2
|-kernel_conv_index_0_etg_0_remont: 2
|-kernel_conv_index_0_etg_1_desc: 2
|-kernel_conv_index_0_etg_1_remont: 2
|-kernel_conv_index_0_etg_2_desc: 2
|-kernel_conv_index_0_etg_2_remont: 2
|-kernel_conv_index_0_etg_3_desc: 2
|-kernel_conv_index_0_etg_3_remont: 2
|-kernel_conv_index_0_etg_4_desc: 2
|-kernel_conv_index_1_etg_0_desc: 2
|-kernel_conv_index_2_etg_0_desc: 2
|-learning_rate: 1e-06
|-nb_conv_etg_0_desc: 3
|-nb_conv_etg_0_remont: 1
|-nb_conv_etg_1_desc: 1
|-nb_conv_etg_1_remont: 1
|-nb_conv_etg_2_desc: 1
|-nb_conv_etg_2_remont: 1
|-nb_conv_etg_3_desc: 1
|-nb_conv_etg_3_remont: 1
|-nb_conv_etg_4_desc: 1
|-nb_pool: 4
|-pool_type_0: max
|-pool_type_1: max
|-pool_type_2: max
|-pool_type_3: max

#Résultat 2e entrainement

Epoch 1/5
12/12 [==============================] - 60s 5s/step - loss: nan - accuracy: 0.8471 - val_loss: nan - val_accuracy: 0.8244
Epoch 2/5
12/12 [==============================] - 8s 695ms/step - loss: nan - accuracy: 0.8471 - val_loss: nan - val_accuracy: 0.8244
Epoch 3/5
12/12 [==============================] - 8s 695ms/step - loss: nan - accuracy: 0.8471 - val_loss: nan - val_accuracy: 0.8244
Epoch 4/5
12/12 [==============================] - 8s 695ms/step - loss: nan - accuracy: 0.8471 - val_loss: nan - val_accuracy: 0.8244
Epoch 5/5
12/12 [==============================] - 8s 699ms/step - loss: nan - accuracy: 0.8471 - val_loss: nan - val_accuracy: 0.8244
Epoch 1/5
12/12 [==============================] - 9s 714ms/step - loss: nan - accuracy: 0.8471 - val_loss: nan - val_accuracy: 0.8244
Epoch 2/5
12/12 [==============================] - 8s 690ms/step - loss: nan - accuracy: 0.8471 - val_loss: nan - val_accuracy: 0.8244
Epoch 3/5
12/12 [==============================] - 8s 697ms/step - loss: nan - accuracy: 0.8471 - val_loss: nan - val_accuracy: 0.8244
Epoch 4/5
12/12 [==============================] - 8s 688ms/step - loss: nan - accuracy: 0.8471 - val_loss: nan - val_accuracy: 0.8244
Epoch 5/5
12/12 [==============================] - 8s 690ms/step - loss: nan - accuracy: 0.8471 - val_loss: nan - val_accuracy: 0.8244
Epoch 1/5
12/12 [==============================] - 9s 712ms/step - loss: nan - accuracy: 0.8449 - val_loss: nan - val_accuracy: 0.8244
Epoch 2/5
12/12 [==============================] - 8s 690ms/step - loss: nan - accuracy: 0.8471 - val_loss: nan - val_accuracy: 0.8244
Epoch 3/5
12/12 [==============================] - 8s 693ms/step - loss: nan - accuracy: 0.8471 - val_loss: nan - val_accuracy: 0.8244
Epoch 4/5
12/12 [==============================] - 8s 688ms/step - loss: nan - accuracy: 0.8471 - val_loss: nan - val_accuracy: 0.8244
Epoch 5/5
12/12 [==============================] - 8s 694ms/step - loss: nan - accuracy: 0.8471 - val_loss: nan - val_accuracy: 0.8244

/usr/local/lib/python3.6/dist-packages/kerastuner/engine/metrics_tracking.py:92: RuntimeWarning: All-NaN axis encountered
  return np.nanmin(values)

Trial complete
Trial summary
|-Trial ID: 629c3fb4b36d9202bccf82981eedeb62
|-Score: 0.8244484066963196
|-Best step: 0
Hyperparameters:
|-activation_conv_index_0_etg_0_desc: selu
|-activation_conv_index_0_etg_0_remont: relu
|-activation_conv_index_0_etg_1_desc: relu
|-activation_conv_index_0_etg_1_remont: relu
|-activation_conv_index_0_etg_2_desc: relu
|-activation_conv_index_0_etg_2_remont: relu
|-activation_conv_index_0_etg_3_desc: relu
|-activation_conv_index_0_etg_3_remont: relu
|-activation_conv_index_0_etg_4_desc: relu
|-beta_1: 1.0325074675237762
|-beta_2: 1.0016033707761187
|-filtre_conv_index_0_etg_0_desc: 100
|-filtre_conv_index_0_etg_0_remont: 1
|-filtre_conv_index_0_etg_1_desc: 1
|-filtre_conv_index_0_etg_1_remont: 1
|-filtre_conv_index_0_etg_2_desc: 1
|-filtre_conv_index_0_etg_2_remont: 1
|-filtre_conv_index_0_etg_3_desc: 1
|-filtre_conv_index_0_etg_3_remont: 1
|-filtre_conv_index_0_etg_4_desc: 1
|-filtre_deconv_etg_0_remont: 1
|-filtre_deconv_etg_1_remont: 1
|-filtre_deconv_etg_2_remont: 1
|-filtre_deconv_etg_3_remont: 1
|-kernel_conv_index_0_etg_0_desc: 2
|-kernel_conv_index_0_etg_0_remont: 2
|-kernel_conv_index_0_etg_1_desc: 2
|-kernel_conv_index_0_etg_1_remont: 2
|-kernel_conv_index_0_etg_2_desc: 2
|-kernel_conv_index_0_etg_2_remont: 2
|-kernel_conv_index_0_etg_3_desc: 2
|-kernel_conv_index_0_etg_3_remont: 2
|-kernel_conv_index_0_etg_4_desc: 2
|-learning_rate: 0.01
|-nb_conv_etg_0_desc: 1
|-nb_conv_etg_0_remont: 1
|-nb_conv_etg_1_desc: 1
|-nb_conv_etg_1_remont: 1
|-nb_conv_etg_2_desc: 1
|-nb_conv_etg_2_remont: 1
|-nb_conv_etg_3_desc: 1
|-nb_conv_etg_3_remont: 1
|-nb_conv_etg_4_desc: 1
|-nb_pool: 4
|-pool_type_0: max
|-pool_type_1: max
|-pool_type_2: max
|-pool_type_3: max

Epoch 1/5
12/12 [==============================] - 11s 886ms/step - loss: 0.0077 - accuracy: 0.6418 - val_loss: 0.0077 - val_accuracy: 0.8244
Epoch 2/5
12/12 [==============================] - 10s 841ms/step - loss: 0.0077 - accuracy: 0.8471 - val_loss: 0.0077 - val_accuracy: 0.8244
Epoch 3/5
12/12 [==============================] - 10s 840ms/step - loss: 0.0077 - accuracy: 0.8471 - val_loss: 0.0077 - val_accuracy: 0.8244
Epoch 4/5
12/12 [==============================] - 10s 834ms/step - loss: 0.0077 - accuracy: 0.8471 - val_loss: 0.0077 - val_accuracy: 0.8244
Epoch 5/5
12/12 [==============================] - 10s 839ms/step - loss: 0.0077 - accuracy: 0.8471 - val_loss: 0.0077 - val_accuracy: 0.8244
Epoch 1/5
12/12 [==============================] - 11s 875ms/step - loss: 0.0077 - accuracy: 0.6418 - val_loss: 0.0077 - val_accuracy: 0.8244
Epoch 2/5
12/12 [==============================] - 10s 840ms/step - loss: 0.0077 - accuracy: 0.8471 - val_loss: 0.0077 - val_accuracy: 0.8244
Epoch 3/5
12/12 [==============================] - 10s 842ms/step - loss: 0.0077 - accuracy: 0.8471 - val_loss: 0.0077 - val_accuracy: 0.8244
Epoch 4/5
12/12 [==============================] - 10s 835ms/step - loss: 0.0077 - accuracy: 0.8471 - val_loss: 0.0077 - val_accuracy: 0.8244
Epoch 5/5
12/12 [==============================] - 10s 834ms/step - loss: 0.0077 - accuracy: 0.8471 - val_loss: 0.0077 - val_accuracy: 0.8244
Epoch 1/5
12/12 [==============================] - 10s 872ms/step - loss: 0.0077 - accuracy: 0.6418 - val_loss: 0.0077 - val_accuracy: 0.8244
Epoch 2/5
12/12 [==============================] - 10s 835ms/step - loss: 0.0077 - accuracy: 0.8471 - val_loss: 0.0077 - val_accuracy: 0.8244
Epoch 3/5
12/12 [==============================] - 10s 851ms/step - loss: 0.0077 - accuracy: 0.8471 - val_loss: 0.0077 - val_accuracy: 0.8244
Epoch 4/5
12/12 [==============================] - 10s 842ms/step - loss: 0.0077 - accuracy: 0.8471 - val_loss: 0.0077 - val_accuracy: 0.8244
Epoch 5/5
12/12 [==============================] - 10s 837ms/step - loss: 0.0077 - accuracy: 0.8471 - val_loss: 0.0077 - val_accuracy: 0.8244

Trial complete
Trial summary
|-Trial ID: 8677c28d78535c9971e72faff9b51789
|-Score: 0.8244484066963196
|-Best step: 0
Hyperparameters:
|-activation_conv_index_0_etg_0_desc: tanh
|-activation_conv_index_0_etg_0_remont: relu
|-activation_conv_index_0_etg_1_desc: elu
|-activation_conv_index_0_etg_1_remont: elu
|-activation_conv_index_0_etg_2_desc: elu
|-activation_conv_index_0_etg_2_remont: elu
|-activation_conv_index_0_etg_3_desc: tanh
|-activation_conv_index_0_etg_3_remont: linear
|-activation_conv_index_0_etg_4_desc: selu
|-activation_conv_index_0_etg_4_remont: relu
|-activation_conv_index_0_etg_5_desc: relu
|-activation_conv_index_0_etg_5_remont: relu
|-activation_conv_index_0_etg_6_desc: relu
|-activation_conv_index_0_etg_6_remont: relu
|-activation_conv_index_0_etg_7_desc: relu
|-activation_conv_index_0_etg_7_remont: relu
|-activation_conv_index_0_etg_8_desc: relu
|-activation_conv_index_1_etg_0_desc: relu
|-activation_conv_index_1_etg_2_desc: relu
|-activation_conv_index_1_etg_2_remont: relu
|-activation_conv_index_1_etg_3_remont: relu
|-activation_conv_index_1_etg_4_desc: relu
|-activation_conv_index_2_etg_0_desc: relu
|-activation_conv_index_2_etg_2_remont: relu
|-activation_conv_index_2_etg_3_remont: relu
|-activation_conv_index_2_etg_4_desc: relu
|-activation_conv_index_3_etg_0_desc: relu
|-beta_1: 1.024879745402238
|-beta_2: 0.9977119751844172
|-filtre_conv_index_0_etg_0_desc: 3
|-filtre_conv_index_0_etg_0_remont: 3
|-filtre_conv_index_0_etg_1_desc: 3
|-filtre_conv_index_0_etg_1_remont: 1
|-filtre_conv_index_0_etg_2_desc: 10
|-filtre_conv_index_0_etg_2_remont: 1
|-filtre_conv_index_0_etg_3_desc: 1
|-filtre_conv_index_0_etg_3_remont: 50
|-filtre_conv_index_0_etg_4_desc: 50
|-filtre_conv_index_0_etg_4_remont: 1
|-filtre_conv_index_0_etg_5_desc: 1
|-filtre_conv_index_0_etg_5_remont: 1
|-filtre_conv_index_0_etg_6_desc: 1
|-filtre_conv_index_0_etg_6_remont: 1
|-filtre_conv_index_0_etg_7_desc: 1
|-filtre_conv_index_0_etg_7_remont: 1
|-filtre_conv_index_0_etg_8_desc: 1
|-filtre_conv_index_1_etg_0_desc: 1
|-filtre_conv_index_1_etg_2_desc: 1
|-filtre_conv_index_1_etg_2_remont: 1
|-filtre_conv_index_1_etg_3_remont: 1
|-filtre_conv_index_1_etg_4_desc: 1
|-filtre_conv_index_2_etg_0_desc: 1
|-filtre_conv_index_2_etg_2_remont: 1
|-filtre_conv_index_2_etg_3_remont: 1
|-filtre_conv_index_2_etg_4_desc: 1
|-filtre_conv_index_3_etg_0_desc: 1
|-filtre_deconv_etg_0_remont: 100
|-filtre_deconv_etg_1_remont: 1
|-filtre_deconv_etg_2_remont: 50
|-filtre_deconv_etg_3_remont: 500
|-filtre_deconv_etg_4_remont: 1
|-filtre_deconv_etg_5_remont: 1
|-filtre_deconv_etg_6_remont: 1
|-filtre_deconv_etg_7_remont: 1
|-kernel_conv_index_0_etg_0_desc: 3
|-kernel_conv_index_0_etg_0_remont: 2
|-kernel_conv_index_0_etg_1_desc: 3
|-kernel_conv_index_0_etg_1_remont: 3
|-kernel_conv_index_0_etg_2_desc: 2
|-kernel_conv_index_0_etg_2_remont: 3
|-kernel_conv_index_0_etg_3_desc: 3
|-kernel_conv_index_0_etg_3_remont: 2
|-kernel_conv_index_0_etg_4_desc: 3
|-kernel_conv_index_0_etg_4_remont: 2
|-kernel_conv_index_0_etg_5_desc: 2
|-kernel_conv_index_0_etg_5_remont: 2
|-kernel_conv_index_0_etg_6_desc: 2
|-kernel_conv_index_0_etg_6_remont: 2
|-kernel_conv_index_0_etg_7_desc: 2
|-kernel_conv_index_0_etg_7_remont: 2
|-kernel_conv_index_0_etg_8_desc: 2
|-kernel_conv_index_1_etg_0_desc: 2
|-kernel_conv_index_1_etg_2_desc: 2
|-kernel_conv_index_1_etg_2_remont: 2
|-kernel_conv_index_1_etg_3_remont: 2
|-kernel_conv_index_1_etg_4_desc: 2
|-kernel_conv_index_2_etg_0_desc: 2
|-kernel_conv_index_2_etg_2_remont: 2
|-kernel_conv_index_2_etg_3_remont: 2
|-kernel_conv_index_2_etg_4_desc: 2
|-kernel_conv_index_3_etg_0_desc: 2
|-learning_rate: 1e-06
|-nb_conv_etg_0_desc: 4
|-nb_conv_etg_0_remont: 1
|-nb_conv_etg_1_desc: 1
|-nb_conv_etg_1_remont: 0
|-nb_conv_etg_2_desc: 2
|-nb_conv_etg_2_remont: 3
|-nb_conv_etg_3_desc: 1
|-nb_conv_etg_3_remont: 3
|-nb_conv_etg_4_desc: 3
|-nb_conv_etg_4_remont: 1
|-nb_conv_etg_5_desc: 1
|-nb_conv_etg_5_remont: 1
|-nb_conv_etg_6_desc: 1
|-nb_conv_etg_6_remont: 1
|-nb_conv_etg_7_desc: 1
|-nb_conv_etg_7_remont: 1
|-nb_conv_etg_8_desc: 1
|-nb_pool: 8
|-pool_type_0: max
|-pool_type_1: max
|-pool_type_2: max
|-pool_type_3: max
|-pool_type_4: max
|-pool_type_5: max
|-pool_type_6: max
|-pool_type_7: max

Epoch 1/5
12/12 [==============================] - 29s 2s/step - loss: nan - accuracy: 0.8447 - val_loss: nan - val_accuracy: 0.8244
Epoch 2/5
12/12 [==============================] - 29s 2s/step - loss: nan - accuracy: 0.8471 - val_loss: nan - val_accuracy: 0.8244
Epoch 3/5
12/12 [==============================] - 29s 2s/step - loss: nan - accuracy: 0.8471 - val_loss: nan - val_accuracy: 0.8244
Epoch 4/5
12/12 [==============================] - 29s 2s/step - loss: nan - accuracy: 0.8471 - val_loss: nan - val_accuracy: 0.8244
Epoch 5/5
12/12 [==============================] - 29s 2s/step - loss: nan - accuracy: 0.8471 - val_loss: nan - val_accuracy: 0.8244
Epoch 1/5
12/12 [==============================] - 29s 2s/step - loss: nan - accuracy: 0.8419 - val_loss: nan - val_accuracy: 0.8244
Epoch 2/5
12/12 [==============================] - 29s 2s/step - loss: nan - accuracy: 0.8471 - val_loss: nan - val_accuracy: 0.8244
Epoch 3/5
12/12 [==============================] - 28s 2s/step - loss: nan - accuracy: 0.8471 - val_loss: nan - val_accuracy: 0.8244
Epoch 4/5
12/12 [==============================] - 29s 2s/step - loss: nan - accuracy: 0.8471 - val_loss: nan - val_accuracy: 0.8244
Epoch 5/5
12/12 [==============================] - 29s 2s/step - loss: nan - accuracy: 0.8471 - val_loss: nan - val_accuracy: 0.8244
Epoch 1/5
12/12 [==============================] - 29s 2s/step - loss: nan - accuracy: 0.8370 - val_loss: nan - val_accuracy: 0.8244
Epoch 2/5
12/12 [==============================] - 29s 2s/step - loss: nan - accuracy: 0.8471 - val_loss: nan - val_accuracy: 0.8244
Epoch 3/5
12/12 [==============================] - 29s 2s/step - loss: nan - accuracy: 0.8471 - val_loss: nan - val_accuracy: 0.8244
Epoch 4/5
12/12 [==============================] - 29s 2s/step - loss: nan - accuracy: 0.8471 - val_loss: nan - val_accuracy: 0.8244
Epoch 5/5
12/12 [==============================] - 29s 2s/step - loss: nan - accuracy: 0.8471 - val_loss: nan - val_accuracy: 0.8244

/usr/local/lib/python3.6/dist-packages/kerastuner/engine/metrics_tracking.py:92: RuntimeWarning: All-NaN axis encountered
  return np.nanmin(values)

Trial complete
Trial summary
|-Trial ID: 3e81974e3e23220061de6259aeba5e96
|-Score: 0.8244484066963196
|-Best step: 0
Hyperparameters:
|-activation_conv_index_0_etg_0_desc: linear
|-activation_conv_index_0_etg_0_remont: selu
|-activation_conv_index_0_etg_1_desc: linear
|-activation_conv_index_0_etg_1_remont: relu
|-activation_conv_index_0_etg_2_desc: selu
|-activation_conv_index_0_etg_2_remont: linear
|-activation_conv_index_0_etg_3_desc: elu
|-activation_conv_index_0_etg_3_remont: tanh
|-activation_conv_index_0_etg_4_desc: relu
|-activation_conv_index_0_etg_4_remont: linear
|-activation_conv_index_0_etg_5_desc: selu
|-activation_conv_index_0_etg_5_remont: linear
|-activation_conv_index_0_etg_6_desc: elu
|-activation_conv_index_0_etg_6_remont: elu
|-activation_conv_index_0_etg_7_desc: elu
|-activation_conv_index_0_etg_7_remont: tanh
|-activation_conv_index_0_etg_8_desc: selu
|-activation_conv_index_1_etg_0_desc: relu
|-activation_conv_index_1_etg_1_desc: relu
|-activation_conv_index_1_etg_2_desc: tanh
|-activation_conv_index_1_etg_2_remont: tanh
|-activation_conv_index_1_etg_3_remont: relu
|-activation_conv_index_1_etg_4_desc: relu
|-activation_conv_index_2_etg_0_desc: selu
|-activation_conv_index_2_etg_1_desc: relu
|-activation_conv_index_2_etg_2_desc: relu
|-activation_conv_index_2_etg_2_remont: relu
|-activation_conv_index_2_etg_3_remont: elu
|-activation_conv_index_2_etg_4_desc: linear
|-activation_conv_index_3_etg_0_desc: tanh
|-beta_1: 1.0479349577513535
|-beta_2: 1.0424958249935516
|-filtre_conv_index_0_etg_0_desc: 10
|-filtre_conv_index_0_etg_0_remont: 3
|-filtre_conv_index_0_etg_1_desc: 100
|-filtre_conv_index_0_etg_1_remont: 1
|-filtre_conv_index_0_etg_2_desc: 10
|-filtre_conv_index_0_etg_2_remont: 10
|-filtre_conv_index_0_etg_3_desc: 10
|-filtre_conv_index_0_etg_3_remont: 500
|-filtre_conv_index_0_etg_4_desc: 50
|-filtre_conv_index_0_etg_4_remont: 50
|-filtre_conv_index_0_etg_5_desc: 10
|-filtre_conv_index_0_etg_5_remont: 1
|-filtre_conv_index_0_etg_6_desc: 1
|-filtre_conv_index_0_etg_6_remont: 500
|-filtre_conv_index_0_etg_7_desc: 100
|-filtre_conv_index_0_etg_7_remont: 3
|-filtre_conv_index_0_etg_8_desc: 100
|-filtre_conv_index_1_etg_0_desc: 100
|-filtre_conv_index_1_etg_1_desc: 1
|-filtre_conv_index_1_etg_2_desc: 50
|-filtre_conv_index_1_etg_2_remont: 10
|-filtre_conv_index_1_etg_3_remont: 500
|-filtre_conv_index_1_etg_4_desc: 50
|-filtre_conv_index_2_etg_0_desc: 100
|-filtre_conv_index_2_etg_1_desc: 1
|-filtre_conv_index_2_etg_2_desc: 1
|-filtre_conv_index_2_etg_2_remont: 500
|-filtre_conv_index_2_etg_3_remont: 500
|-filtre_conv_index_2_etg_4_desc: 3
|-filtre_conv_index_3_etg_0_desc: 50
|-filtre_deconv_etg_0_remont: 100
|-filtre_deconv_etg_1_remont: 10
|-filtre_deconv_etg_2_remont: 500
|-filtre_deconv_etg_3_remont: 100
|-filtre_deconv_etg_4_remont: 50
|-filtre_deconv_etg_5_remont: 3
|-filtre_deconv_etg_6_remont: 10
|-filtre_deconv_etg_7_remont: 500
|-kernel_conv_index_0_etg_0_desc: 2
|-kernel_conv_index_0_etg_0_remont: 3
|-kernel_conv_index_0_etg_1_desc: 2
|-kernel_conv_index_0_etg_1_remont: 2
|-kernel_conv_index_0_etg_2_desc: 2
|-kernel_conv_index_0_etg_2_remont: 3
|-kernel_conv_index_0_etg_3_desc: 3
|-kernel_conv_index_0_etg_3_remont: 3
|-kernel_conv_index_0_etg_4_desc: 2
|-kernel_conv_index_0_etg_4_remont: 2
|-kernel_conv_index_0_etg_5_desc: 2
|-kernel_conv_index_0_etg_5_remont: 3
|-kernel_conv_index_0_etg_6_desc: 3
|-kernel_conv_index_0_etg_6_remont: 2
|-kernel_conv_index_0_etg_7_desc: 2
|-kernel_conv_index_0_etg_7_remont: 2
|-kernel_conv_index_0_etg_8_desc: 2
|-kernel_conv_index_1_etg_0_desc: 3
|-kernel_conv_index_1_etg_1_desc: 2
|-kernel_conv_index_1_etg_2_desc: 3
|-kernel_conv_index_1_etg_2_remont: 2
|-kernel_conv_index_1_etg_3_remont: 3
|-kernel_conv_index_1_etg_4_desc: 2
|-kernel_conv_index_2_etg_0_desc: 3
|-kernel_conv_index_2_etg_1_desc: 2
|-kernel_conv_index_2_etg_2_desc: 2
|-kernel_conv_index_2_etg_2_remont: 3
|-kernel_conv_index_2_etg_3_remont: 3
|-kernel_conv_index_2_etg_4_desc: 3
|-kernel_conv_index_3_etg_0_desc: 2
|-learning_rate: 0.01
|-nb_conv_etg_0_desc: 4
|-nb_conv_etg_0_remont: 1
|-nb_conv_etg_1_desc: 3
|-nb_conv_etg_1_remont: 0
|-nb_conv_etg_2_desc: 3
|-nb_conv_etg_2_remont: 3
|-nb_conv_etg_3_desc: 1
|-nb_conv_etg_3_remont: 3
|-nb_conv_etg_4_desc: 1
|-nb_conv_etg_4_remont: 0
|-nb_conv_etg_5_desc: 2
|-nb_conv_etg_5_remont: 2
|-nb_conv_etg_6_desc: 0
|-nb_conv_etg_6_remont: 1
|-nb_conv_etg_7_desc: 2
|-nb_conv_etg_7_remont: 2
|-nb_conv_etg_8_desc: 1
|-nb_pool: 2
|-pool_type_0: avg
|-pool_type_1: avg
|-pool_type_2: avg
|-pool_type_3: avg
|-pool_type_4: avg
|-pool_type_5: max
|-pool_type_6: max
|-pool_type_7: avg

Epoch 1/5
12/12 [==============================] - 515s 43s/step - loss: 0.0076 - accuracy: 0.8358 - val_loss: 0.0074 - val_accuracy: 0.8244
Epoch 2/5
12/12 [==============================] - 517s 43s/step - loss: 0.0073 - accuracy: 0.8471 - val_loss: 0.0071 - val_accuracy: 0.8244
Epoch 3/5
12/12 [==============================] - 518s 43s/step - loss: 0.0070 - accuracy: 0.8471 - val_loss: 0.0068 - val_accuracy: 0.8244
Epoch 4/5
12/12 [==============================] - 515s 43s/step - loss: 0.0067 - accuracy: 0.8471 - val_loss: 0.0065 - val_accuracy: 0.8244
Epoch 5/5
12/12 [==============================] - 515s 43s/step - loss: 0.0064 - accuracy: 0.8471 - val_loss: 0.0062 - val_accuracy: 0.8244
Epoch 1/5
12/12 [==============================] - 514s 43s/step - loss: 0.0076 - accuracy: 0.8044 - val_loss: 0.0074 - val_accuracy: 0.8207
Epoch 2/5
12/12 [==============================] - 513s 43s/step - loss: 0.0073 - accuracy: 0.8433 - val_loss: 0.0071 - val_accuracy: 0.8207
Epoch 3/5
12/12 [==============================] - 513s 43s/step - loss: 0.0071 - accuracy: 0.8433 - val_loss: 0.0068 - val_accuracy: 0.8207
Epoch 4/5
12/12 [==============================] - 513s 43s/step - loss: 0.0068 - accuracy: 0.8433 - val_loss: 0.0066 - val_accuracy: 0.8207
Epoch 5/5
12/12 [==============================] - 514s 43s/step - loss: 0.0065 - accuracy: 0.8433 - val_loss: 0.0063 - val_accuracy: 0.8207
Epoch 1/5
12/12 [==============================] - 514s 43s/step - loss: 0.0076 - accuracy: 0.8272 - val_loss: 0.0074 - val_accuracy: 0.8244
Epoch 2/5
12/12 [==============================] - 514s 43s/step - loss: 0.0073 - accuracy: 0.8471 - val_loss: 0.0071 - val_accuracy: 0.8244
Epoch 3/5
12/12 [==============================] - 512s 43s/step - loss: 0.0070 - accuracy: 0.8471 - val_loss: 0.0068 - val_accuracy: 0.8244
Epoch 4/5
12/12 [==============================] - 516s 43s/step - loss: 0.0067 - accuracy: 0.8471 - val_loss: 0.0065 - val_accuracy: 0.8244
Epoch 5/5
12/12 [==============================] - 514s 43s/step - loss: 0.0065 - accuracy: 0.8471 - val_loss: 0.0062 - val_accuracy: 0.8244

Trial complete
Trial summary
|-Trial ID: fa603dc005b20ee7474cbae8e86a8aed
|-Score: 0.8231907288233439
|-Best step: 0
Hyperparameters:
|-activation_conv_index_0_etg_0_desc: elu
|-activation_conv_index_0_etg_0_remont: elu
|-activation_conv_index_0_etg_1_desc: selu
|-activation_conv_index_0_etg_1_remont: relu
|-activation_conv_index_0_etg_2_desc: elu
|-activation_conv_index_0_etg_2_remont: elu
|-activation_conv_index_0_etg_3_desc: linear
|-activation_conv_index_0_etg_3_remont: relu
|-activation_conv_index_0_etg_4_desc: tanh
|-activation_conv_index_0_etg_4_remont: relu
|-activation_conv_index_0_etg_5_desc: tanh
|-activation_conv_index_0_etg_5_remont: elu
|-activation_conv_index_0_etg_6_desc: relu
|-activation_conv_index_0_etg_6_remont: selu
|-activation_conv_index_0_etg_7_desc: linear
|-activation_conv_index_0_etg_7_remont: linear
|-activation_conv_index_0_etg_8_desc: relu
|-activation_conv_index_1_etg_0_desc: selu
|-activation_conv_index_1_etg_1_desc: selu
|-activation_conv_index_1_etg_1_remont: relu
|-activation_conv_index_1_etg_2_desc: selu
|-activation_conv_index_1_etg_2_remont: linear
|-activation_conv_index_1_etg_3_desc: relu
|-activation_conv_index_1_etg_3_remont: elu
|-activation_conv_index_1_etg_4_desc: relu
|-activation_conv_index_2_etg_0_desc: elu
|-activation_conv_index_2_etg_1_desc: elu
|-activation_conv_index_2_etg_2_desc: selu
|-activation_conv_index_2_etg_2_remont: tanh
|-activation_conv_index_2_etg_3_desc: relu
|-activation_conv_index_2_etg_3_remont: tanh
|-activation_conv_index_2_etg_4_desc: relu
|-activation_conv_index_3_etg_0_desc: relu
|-beta_1: 1.0856655596594058
|-beta_2: 0.9111979255753918
|-filtre_conv_index_0_etg_0_desc: 10
|-filtre_conv_index_0_etg_0_remont: 500
|-filtre_conv_index_0_etg_1_desc: 50
|-filtre_conv_index_0_etg_1_remont: 10
|-filtre_conv_index_0_etg_2_desc: 100
|-filtre_conv_index_0_etg_2_remont: 3
|-filtre_conv_index_0_etg_3_desc: 50
|-filtre_conv_index_0_etg_3_remont: 100
|-filtre_conv_index_0_etg_4_desc: 500
|-filtre_conv_index_0_etg_4_remont: 50
|-filtre_conv_index_0_etg_5_desc: 10
|-filtre_conv_index_0_etg_5_remont: 10
|-filtre_conv_index_0_etg_6_desc: 3
|-filtre_conv_index_0_etg_6_remont: 50
|-filtre_conv_index_0_etg_7_desc: 1
|-filtre_conv_index_0_etg_7_remont: 1
|-filtre_conv_index_0_etg_8_desc: 50
|-filtre_conv_index_1_etg_0_desc: 3
|-filtre_conv_index_1_etg_1_desc: 100
|-filtre_conv_index_1_etg_1_remont: 1
|-filtre_conv_index_1_etg_2_desc: 3
|-filtre_conv_index_1_etg_2_remont: 3
|-filtre_conv_index_1_etg_3_desc: 1
|-filtre_conv_index_1_etg_3_remont: 500
|-filtre_conv_index_1_etg_4_desc: 500
|-filtre_conv_index_2_etg_0_desc: 500
|-filtre_conv_index_2_etg_1_desc: 1
|-filtre_conv_index_2_etg_2_desc: 50
|-filtre_conv_index_2_etg_2_remont: 100
|-filtre_conv_index_2_etg_3_desc: 1
|-filtre_conv_index_2_etg_3_remont: 10
|-filtre_conv_index_2_etg_4_desc: 100
|-filtre_conv_index_3_etg_0_desc: 10
|-filtre_deconv_etg_0_remont: 50
|-filtre_deconv_etg_1_remont: 1
|-filtre_deconv_etg_2_remont: 1
|-filtre_deconv_etg_3_remont: 100
|-filtre_deconv_etg_4_remont: 50
|-filtre_deconv_etg_5_remont: 50
|-filtre_deconv_etg_6_remont: 1
|-filtre_deconv_etg_7_remont: 1
|-kernel_conv_index_0_etg_0_desc: 3
|-kernel_conv_index_0_etg_0_remont: 3
|-kernel_conv_index_0_etg_1_desc: 3
|-kernel_conv_index_0_etg_1_remont: 3
|-kernel_conv_index_0_etg_2_desc: 3
|-kernel_conv_index_0_etg_2_remont: 2
|-kernel_conv_index_0_etg_3_desc: 3
|-kernel_conv_index_0_etg_3_remont: 3
|-kernel_conv_index_0_etg_4_desc: 3
|-kernel_conv_index_0_etg_4_remont: 2
|-kernel_conv_index_0_etg_5_desc: 3
|-kernel_conv_index_0_etg_5_remont: 2
|-kernel_conv_index_0_etg_6_desc: 2
|-kernel_conv_index_0_etg_6_remont: 2
|-kernel_conv_index_0_etg_7_desc: 2
|-kernel_conv_index_0_etg_7_remont: 3
|-kernel_conv_index_0_etg_8_desc: 2
|-kernel_conv_index_1_etg_0_desc: 3
|-kernel_conv_index_1_etg_1_desc: 2
|-kernel_conv_index_1_etg_1_remont: 2
|-kernel_conv_index_1_etg_2_desc: 3
|-kernel_conv_index_1_etg_2_remont: 3
|-kernel_conv_index_1_etg_3_desc: 2
|-kernel_conv_index_1_etg_3_remont: 2
|-kernel_conv_index_1_etg_4_desc: 2
|-kernel_conv_index_2_etg_0_desc: 3
|-kernel_conv_index_2_etg_1_desc: 2
|-kernel_conv_index_2_etg_2_desc: 3
|-kernel_conv_index_2_etg_2_remont: 3
|-kernel_conv_index_2_etg_3_desc: 2
|-kernel_conv_index_2_etg_3_remont: 3
|-kernel_conv_index_2_etg_4_desc: 2
|-kernel_conv_index_3_etg_0_desc: 2
|-learning_rate: 1e-05
|-nb_conv_etg_0_desc: 1
|-nb_conv_etg_0_remont: 0
|-nb_conv_etg_1_desc: 2
|-nb_conv_etg_1_remont: 2
|-nb_conv_etg_2_desc: 1
|-nb_conv_etg_2_remont: 2
|-nb_conv_etg_3_desc: 3
|-nb_conv_etg_3_remont: 2
|-nb_conv_etg_4_desc: 3
|-nb_conv_etg_4_remont: 0
|-nb_conv_etg_5_desc: 2
|-nb_conv_etg_5_remont: 4
|-nb_conv_etg_6_desc: 0
|-nb_conv_etg_6_remont: 3
|-nb_conv_etg_7_desc: 1
|-nb_conv_etg_7_remont: 2
|-nb_conv_etg_8_desc: 1
|-nb_pool: 4
|-pool_type_0: avg
|-pool_type_1: max
|-pool_type_2: avg
|-pool_type_3: max
|-pool_type_4: avg
|-pool_type_5: avg
|-pool_type_6: avg
|-pool_type_7: max

Epoch 1/5
12/12 [==============================] - 32s 3s/step - loss: nan - accuracy: 0.7878 - val_loss: nan - val_accuracy: 0.8244
Epoch 2/5
12/12 [==============================] - 32s 3s/step - loss: nan - accuracy: 0.8471 - val_loss: nan - val_accuracy: 0.8244
Epoch 3/5
12/12 [==============================] - 31s 3s/step - loss: nan - accuracy: 0.8471 - val_loss: nan - val_accuracy: 0.8244
Epoch 4/5
12/12 [==============================] - 32s 3s/step - loss: nan - accuracy: 0.8471 - val_loss: nan - val_accuracy: 0.8244
Epoch 5/5
12/12 [==============================] - 32s 3s/step - loss: nan - accuracy: 0.8471 - val_loss: nan - val_accuracy: 0.8244
Epoch 1/5
12/12 [==============================] - 32s 3s/step - loss: nan - accuracy: 0.7996 - val_loss: nan - val_accuracy: 0.8244
Epoch 2/5
12/12 [==============================] - 31s 3s/step - loss: nan - accuracy: 0.8471 - val_loss: nan - val_accuracy: 0.8244
Epoch 3/5
12/12 [==============================] - 32s 3s/step - loss: nan - accuracy: 0.8471 - val_loss: nan - val_accuracy: 0.8244
Epoch 4/5
12/12 [==============================] - 32s 3s/step - loss: nan - accuracy: 0.8471 - val_loss: nan - val_accuracy: 0.8244
Epoch 5/5
12/12 [==============================] - 32s 3s/step - loss: nan - accuracy: 0.8471 - val_loss: nan - val_accuracy: 0.8244
Epoch 1/5
12/12 [==============================] - 32s 3s/step - loss: nan - accuracy: 0.7904 - val_loss: nan - val_accuracy: 0.8244
Epoch 2/5
12/12 [==============================] - 32s 3s/step - loss: nan - accuracy: 0.8471 - val_loss: nan - val_accuracy: 0.8244
Epoch 3/5
12/12 [==============================] - 32s 3s/step - loss: nan - accuracy: 0.8471 - val_loss: nan - val_accuracy: 0.8244
Epoch 4/5
12/12 [==============================] - 32s 3s/step - loss: nan - accuracy: 0.8471 - val_loss: nan - val_accuracy: 0.8244
Epoch 5/5
12/12 [==============================] - 32s 3s/step - loss: nan - accuracy: 0.8471 - val_loss: nan - val_accuracy: 0.8244

/usr/local/lib/python3.6/dist-packages/kerastuner/engine/metrics_tracking.py:92: RuntimeWarning: All-NaN axis encountered
  return np.nanmin(values)

Trial complete
Trial summary
|-Trial ID: 96747948f0edc64bc9730f56084caffe
|-Score: 0.8244484066963196
|-Best step: 0
Hyperparameters:
|-activation_conv_index_0_etg_0_desc: tanh
|-activation_conv_index_0_etg_0_remont: selu
|-activation_conv_index_0_etg_1_desc: linear
|-activation_conv_index_0_etg_1_remont: relu
|-activation_conv_index_0_etg_2_desc: linear
|-activation_conv_index_0_etg_2_remont: selu
|-activation_conv_index_0_etg_3_desc: relu
|-activation_conv_index_0_etg_3_remont: elu
|-activation_conv_index_0_etg_4_desc: relu
|-activation_conv_index_0_etg_4_remont: tanh
|-activation_conv_index_0_etg_5_desc: tanh
|-activation_conv_index_0_etg_5_remont: elu
|-activation_conv_index_0_etg_6_desc: relu
|-activation_conv_index_0_etg_6_remont: linear
|-activation_conv_index_0_etg_7_desc: linear
|-activation_conv_index_0_etg_7_remont: elu
|-activation_conv_index_0_etg_8_desc: selu
|-activation_conv_index_1_etg_0_desc: tanh
|-activation_conv_index_1_etg_1_desc: elu
|-activation_conv_index_1_etg_1_remont: selu
|-activation_conv_index_1_etg_2_desc: tanh
|-activation_conv_index_1_etg_2_remont: selu
|-activation_conv_index_1_etg_3_desc: elu
|-activation_conv_index_1_etg_3_remont: tanh
|-activation_conv_index_1_etg_4_desc: relu
|-activation_conv_index_2_etg_0_desc: tanh
|-activation_conv_index_2_etg_1_desc: relu
|-activation_conv_index_2_etg_2_desc: tanh
|-activation_conv_index_2_etg_2_remont: selu
|-activation_conv_index_2_etg_3_desc: relu
|-activation_conv_index_2_etg_3_remont: linear
|-activation_conv_index_2_etg_4_desc: elu
|-activation_conv_index_3_etg_0_desc: elu
|-beta_1: 0.9145705889012196
|-beta_2: 1.1573236362544304
|-filtre_conv_index_0_etg_0_desc: 50
|-filtre_conv_index_0_etg_0_remont: 10
|-filtre_conv_index_0_etg_1_desc: 3
|-filtre_conv_index_0_etg_1_remont: 10
|-filtre_conv_index_0_etg_2_desc: 500
|-filtre_conv_index_0_etg_2_remont: 50
|-filtre_conv_index_0_etg_3_desc: 100
|-filtre_conv_index_0_etg_3_remont: 500
|-filtre_conv_index_0_etg_4_desc: 100
|-filtre_conv_index_0_etg_4_remont: 10
|-filtre_conv_index_0_etg_5_desc: 10
|-filtre_conv_index_0_etg_5_remont: 50
|-filtre_conv_index_0_etg_6_desc: 3
|-filtre_conv_index_0_etg_6_remont: 50
|-filtre_conv_index_0_etg_7_desc: 100
|-filtre_conv_index_0_etg_7_remont: 3
|-filtre_conv_index_0_etg_8_desc: 1
|-filtre_conv_index_1_etg_0_desc: 10
|-filtre_conv_index_1_etg_1_desc: 1
|-filtre_conv_index_1_etg_1_remont: 1
|-filtre_conv_index_1_etg_2_desc: 3
|-filtre_conv_index_1_etg_2_remont: 500
|-filtre_conv_index_1_etg_3_desc: 500
|-filtre_conv_index_1_etg_3_remont: 10
|-filtre_conv_index_1_etg_4_desc: 500
|-filtre_conv_index_2_etg_0_desc: 50
|-filtre_conv_index_2_etg_1_desc: 10
|-filtre_conv_index_2_etg_2_desc: 100
|-filtre_conv_index_2_etg_2_remont: 3
|-filtre_conv_index_2_etg_3_desc: 10
|-filtre_conv_index_2_etg_3_remont: 500
|-filtre_conv_index_2_etg_4_desc: 1
|-filtre_conv_index_3_etg_0_desc: 3
|-filtre_deconv_etg_0_remont: 100
|-filtre_deconv_etg_1_remont: 500
|-filtre_deconv_etg_2_remont: 1
|-filtre_deconv_etg_3_remont: 50
|-filtre_deconv_etg_4_remont: 50
|-filtre_deconv_etg_5_remont: 1
|-filtre_deconv_etg_6_remont: 1
|-filtre_deconv_etg_7_remont: 10
|-kernel_conv_index_0_etg_0_desc: 2
|-kernel_conv_index_0_etg_0_remont: 3
|-kernel_conv_index_0_etg_1_desc: 2
|-kernel_conv_index_0_etg_1_remont: 2
|-kernel_conv_index_0_etg_2_desc: 2
|-kernel_conv_index_0_etg_2_remont: 2
|-kernel_conv_index_0_etg_3_desc: 3
|-kernel_conv_index_0_etg_3_remont: 2
|-kernel_conv_index_0_etg_4_desc: 2
|-kernel_conv_index_0_etg_4_remont: 3
|-kernel_conv_index_0_etg_5_desc: 2
|-kernel_conv_index_0_etg_5_remont: 2
|-kernel_conv_index_0_etg_6_desc: 3
|-kernel_conv_index_0_etg_6_remont: 3
|-kernel_conv_index_0_etg_7_desc: 3
|-kernel_conv_index_0_etg_7_remont: 2
|-kernel_conv_index_0_etg_8_desc: 3
|-kernel_conv_index_1_etg_0_desc: 2
|-kernel_conv_index_1_etg_1_desc: 3
|-kernel_conv_index_1_etg_1_remont: 2
|-kernel_conv_index_1_etg_2_desc: 3
|-kernel_conv_index_1_etg_2_remont: 3
|-kernel_conv_index_1_etg_3_desc: 2
|-kernel_conv_index_1_etg_3_remont: 2
|-kernel_conv_index_1_etg_4_desc: 2
|-kernel_conv_index_2_etg_0_desc: 2
|-kernel_conv_index_2_etg_1_desc: 2
|-kernel_conv_index_2_etg_2_desc: 2
|-kernel_conv_index_2_etg_2_remont: 2
|-kernel_conv_index_2_etg_3_desc: 3
|-kernel_conv_index_2_etg_3_remont: 2
|-kernel_conv_index_2_etg_4_desc: 3
|-kernel_conv_index_3_etg_0_desc: 2
|-learning_rate: 0.01
|-nb_conv_etg_0_desc: 0
|-nb_conv_etg_0_remont: 0
|-nb_conv_etg_1_desc: 2
|-nb_conv_etg_1_remont: 2
|-nb_conv_etg_2_desc: 0
|-nb_conv_etg_2_remont: 3
|-nb_conv_etg_3_desc: 2
|-nb_conv_etg_3_remont: 1
|-nb_conv_etg_4_desc: 3
|-nb_conv_etg_4_remont: 3
|-nb_conv_etg_5_desc: 0
|-nb_conv_etg_5_remont: 2
|-nb_conv_etg_6_desc: 3
|-nb_conv_etg_6_remont: 2
|-nb_conv_etg_7_desc: 0
|-nb_conv_etg_7_remont: 1
|-nb_conv_etg_8_desc: 2
|-nb_pool: 1
|-pool_type_0: avg
|-pool_type_1: avg
|-pool_type_2: avg
|-pool_type_3: max
|-pool_type_4: avg
|-pool_type_5: max
|-pool_type_6: max
|-pool_type_7: max

Epoch 1/5
12/12 [==============================] - 652s 54s/step - loss: 0.0077 - accuracy: 0.5680 - val_loss: 0.0077 - val_accuracy: 0.8244
Epoch 2/5
12/12 [==============================] - 652s 54s/step - loss: 0.0077 - accuracy: 0.8471 - val_loss: 0.0076 - val_accuracy: 0.8244
Epoch 3/5
12/12 [==============================] - 651s 54s/step - loss: 0.0077 - accuracy: 0.8471 - val_loss: 0.0076 - val_accuracy: 0.8244
Epoch 4/5
12/12 [==============================] - 656s 55s/step - loss: 0.0077 - accuracy: 0.8471 - val_loss: 0.0076 - val_accuracy: 0.8244
Epoch 5/5
12/12 [==============================] - 659s 55s/step - loss: 0.0077 - accuracy: 0.8471 - val_loss: 0.0076 - val_accuracy: 0.8244
Epoch 1/5
12/12 [==============================] - 651s 54s/step - loss: 0.0077 - accuracy: 0.5957 - val_loss: 0.0077 - val_accuracy: 0.8244
Epoch 2/5
12/12 [==============================] - 658s 55s/step - loss: 0.0077 - accuracy: 0.8471 - val_loss: 0.0076 - val_accuracy: 0.8244
Epoch 3/5
12/12 [==============================] - 654s 54s/step - loss: 0.0077 - accuracy: 0.8471 - val_loss: 0.0076 - val_accuracy: 0.8244
Epoch 4/5
12/12 [==============================] - 655s 55s/step - loss: 0.0077 - accuracy: 0.8471 - val_loss: 0.0076 - val_accuracy: 0.8244
Epoch 5/5
12/12 [==============================] - 655s 55s/step - loss: 0.0077 - accuracy: 0.8471 - val_loss: 0.0076 - val_accuracy: 0.8244
Epoch 1/5
12/12 [==============================] - 651s 54s/step - loss: 0.0077 - accuracy: 0.6259 - val_loss: 0.0077 - val_accuracy: 0.8244
Epoch 2/5
12/12 [==============================] - 650s 54s/step - loss: 0.0077 - accuracy: 0.8471 - val_loss: 0.0076 - val_accuracy: 0.8244
Epoch 3/5
12/12 [==============================] - 652s 54s/step - loss: 0.0077 - accuracy: 0.8471 - val_loss: 0.0076 - val_accuracy: 0.8244
Epoch 4/5
12/12 [==============================] - 651s 54s/step - loss: 0.0077 - accuracy: 0.8471 - val_loss: 0.0076 - val_accuracy: 0.8244
Epoch 5/5
12/12 [==============================] - 653s 54s/step - loss: 0.0077 - accuracy: 0.8471 - val_loss: 0.0076 - val_accuracy: 0.8244

Trial complete
Trial summary
|-Trial ID: 2fa98789a6d7c3462f29db45a4e0d91b
|-Score: 0.8244484066963196
|-Best step: 0
Hyperparameters:
|-activation_conv_index_0_etg_0_desc: selu
|-activation_conv_index_0_etg_0_remont: linear
|-activation_conv_index_0_etg_1_desc: elu
|-activation_conv_index_0_etg_1_remont: selu
|-activation_conv_index_0_etg_2_desc: linear
|-activation_conv_index_0_etg_2_remont: linear
|-activation_conv_index_0_etg_3_desc: selu
|-activation_conv_index_0_etg_3_remont: linear
|-activation_conv_index_0_etg_4_desc: tanh
|-activation_conv_index_0_etg_4_remont: selu
|-activation_conv_index_0_etg_5_desc: tanh
|-activation_conv_index_0_etg_5_remont: selu
|-activation_conv_index_0_etg_6_desc: linear
|-activation_conv_index_0_etg_6_remont: elu
|-activation_conv_index_0_etg_7_desc: selu
|-activation_conv_index_0_etg_7_remont: relu
|-activation_conv_index_0_etg_8_desc: tanh
|-activation_conv_index_1_etg_0_desc: tanh
|-activation_conv_index_1_etg_1_desc: relu
|-activation_conv_index_1_etg_1_remont: linear
|-activation_conv_index_1_etg_2_desc: relu
|-activation_conv_index_1_etg_2_remont: tanh
|-activation_conv_index_1_etg_3_desc: selu
|-activation_conv_index_1_etg_3_remont: relu
|-activation_conv_index_1_etg_4_desc: tanh
|-activation_conv_index_2_etg_0_desc: relu
|-activation_conv_index_2_etg_1_desc: linear
|-activation_conv_index_2_etg_1_remont: relu
|-activation_conv_index_2_etg_2_desc: relu
|-activation_conv_index_2_etg_2_remont: selu
|-activation_conv_index_2_etg_3_desc: linear
|-activation_conv_index_2_etg_3_remont: relu
|-activation_conv_index_2_etg_4_desc: tanh
|-activation_conv_index_3_etg_0_desc: relu
|-activation_conv_index_3_etg_1_remont: relu
|-beta_1: 1.0973459377425627
|-beta_2: 0.8853963530060973
|-filtre_conv_index_0_etg_0_desc: 3
|-filtre_conv_index_0_etg_0_remont: 500
|-filtre_conv_index_0_etg_1_desc: 100
|-filtre_conv_index_0_etg_1_remont: 1
|-filtre_conv_index_0_etg_2_desc: 10
|-filtre_conv_index_0_etg_2_remont: 10
|-filtre_conv_index_0_etg_3_desc: 500
|-filtre_conv_index_0_etg_3_remont: 50
|-filtre_conv_index_0_etg_4_desc: 3
|-filtre_conv_index_0_etg_4_remont: 10
|-filtre_conv_index_0_etg_5_desc: 500
|-filtre_conv_index_0_etg_5_remont: 500
|-filtre_conv_index_0_etg_6_desc: 500
|-filtre_conv_index_0_etg_6_remont: 10
|-filtre_conv_index_0_etg_7_desc: 3
|-filtre_conv_index_0_etg_7_remont: 500
|-filtre_conv_index_0_etg_8_desc: 50
|-filtre_conv_index_1_etg_0_desc: 3
|-filtre_conv_index_1_etg_1_desc: 500
|-filtre_conv_index_1_etg_1_remont: 50
|-filtre_conv_index_1_etg_2_desc: 10
|-filtre_conv_index_1_etg_2_remont: 1
|-filtre_conv_index_1_etg_3_desc: 10
|-filtre_conv_index_1_etg_3_remont: 3
|-filtre_conv_index_1_etg_4_desc: 50
|-filtre_conv_index_2_etg_0_desc: 100
|-filtre_conv_index_2_etg_1_desc: 3
|-filtre_conv_index_2_etg_1_remont: 1
|-filtre_conv_index_2_etg_2_desc: 10
|-filtre_conv_index_2_etg_2_remont: 10
|-filtre_conv_index_2_etg_3_desc: 100
|-filtre_conv_index_2_etg_3_remont: 10
|-filtre_conv_index_2_etg_4_desc: 3
|-filtre_conv_index_3_etg_0_desc: 1
|-filtre_conv_index_3_etg_1_remont: 1
|-filtre_deconv_etg_0_remont: 100
|-filtre_deconv_etg_1_remont: 500
|-filtre_deconv_etg_2_remont: 500
|-filtre_deconv_etg_3_remont: 500
|-filtre_deconv_etg_4_remont: 3
|-filtre_deconv_etg_5_remont: 50
|-filtre_deconv_etg_6_remont: 1
|-filtre_deconv_etg_7_remont: 100
|-kernel_conv_index_0_etg_0_desc: 2
|-kernel_conv_index_0_etg_0_remont: 3
|-kernel_conv_index_0_etg_1_desc: 2
|-kernel_conv_index_0_etg_1_remont: 2
|-kernel_conv_index_0_etg_2_desc: 2
|-kernel_conv_index_0_etg_2_remont: 3
|-kernel_conv_index_0_etg_3_desc: 3
|-kernel_conv_index_0_etg_3_remont: 3
|-kernel_conv_index_0_etg_4_desc: 2
|-kernel_conv_index_0_etg_4_remont: 3
|-kernel_conv_index_0_etg_5_desc: 2
|-kernel_conv_index_0_etg_5_remont: 3
|-kernel_conv_index_0_etg_6_desc: 3
|-kernel_conv_index_0_etg_6_remont: 2
|-kernel_conv_index_0_etg_7_desc: 2
|-kernel_conv_index_0_etg_7_remont: 2
|-kernel_conv_index_0_etg_8_desc: 3
|-kernel_conv_index_1_etg_0_desc: 2
|-kernel_conv_index_1_etg_1_desc: 2
|-kernel_conv_index_1_etg_1_remont: 2
|-kernel_conv_index_1_etg_2_desc: 3
|-kernel_conv_index_1_etg_2_remont: 2
|-kernel_conv_index_1_etg_3_desc: 3
|-kernel_conv_index_1_etg_3_remont: 3
|-kernel_conv_index_1_etg_4_desc: 3
|-kernel_conv_index_2_etg_0_desc: 3
|-kernel_conv_index_2_etg_1_desc: 3
|-kernel_conv_index_2_etg_1_remont: 2
|-kernel_conv_index_2_etg_2_desc: 3
|-kernel_conv_index_2_etg_2_remont: 3
|-kernel_conv_index_2_etg_3_desc: 2
|-kernel_conv_index_2_etg_3_remont: 3
|-kernel_conv_index_2_etg_4_desc: 2
|-kernel_conv_index_3_etg_0_desc: 3
|-kernel_conv_index_3_etg_1_remont: 2
|-learning_rate: 1e-05
|-nb_conv_etg_0_desc: 2
|-nb_conv_etg_0_remont: 1
|-nb_conv_etg_1_desc: 3
|-nb_conv_etg_1_remont: 4
|-nb_conv_etg_2_desc: 3
|-nb_conv_etg_2_remont: 4
|-nb_conv_etg_3_desc: 2
|-nb_conv_etg_3_remont: 1
|-nb_conv_etg_4_desc: 2
|-nb_conv_etg_4_remont: 1
|-nb_conv_etg_5_desc: 1
|-nb_conv_etg_5_remont: 3
|-nb_conv_etg_6_desc: 3
|-nb_conv_etg_6_remont: 2
|-nb_conv_etg_7_desc: 1
|-nb_conv_etg_7_remont: 0
|-nb_conv_etg_8_desc: 4
|-nb_pool: 2
|-pool_type_0: avg
|-pool_type_1: max
|-pool_type_2: avg
|-pool_type_3: avg
|-pool_type_4: max
|-pool_type_5: avg
|-pool_type_6: avg
|-pool_type_7: max

Epoch 1/5
12/12 [==============================] - 169s 14s/step - loss: nan - accuracy: 0.8350 - val_loss: nan - val_accuracy: 0.8244
Epoch 2/5
12/12 [==============================] - 167s 14s/step - loss: nan - accuracy: 0.8471 - val_loss: nan - val_accuracy: 0.8244
Epoch 3/5
12/12 [==============================] - 167s 14s/step - loss: nan - accuracy: 0.8471 - val_loss: nan - val_accuracy: 0.8244
Epoch 4/5
12/12 [==============================] - 167s 14s/step - loss: nan - accuracy: 0.8471 - val_loss: nan - val_accuracy: 0.8244
Epoch 5/5
12/12 [==============================] - 168s 14s/step - loss: nan - accuracy: 0.8471 - val_loss: nan - val_accuracy: 0.8244
Epoch 1/5
12/12 [==============================] - 168s 14s/step - loss: nan - accuracy: 0.8022 - val_loss: nan - val_accuracy: 0.8244
Epoch 2/5
12/12 [==============================] - 167s 14s/step - loss: nan - accuracy: 0.8471 - val_loss: nan - val_accuracy: 0.8244
Epoch 3/5
12/12 [==============================] - 167s 14s/step - loss: nan - accuracy: 0.8471 - val_loss: nan - val_accuracy: 0.8244
Epoch 4/5
12/12 [==============================] - 166s 14s/step - loss: nan - accuracy: 0.8471 - val_loss: nan - val_accuracy: 0.8244
Epoch 5/5
12/12 [==============================] - 167s 14s/step - loss: nan - accuracy: 0.8471 - val_loss: nan - val_accuracy: 0.8244
Epoch 1/5
12/12 [==============================] - 167s 14s/step - loss: nan - accuracy: 0.8311 - val_loss: nan - val_accuracy: 0.8244
Epoch 2/5
12/12 [==============================] - 167s 14s/step - loss: nan - accuracy: 0.8471 - val_loss: nan - val_accuracy: 0.8244
Epoch 3/5
12/12 [==============================] - 166s 14s/step - loss: nan - accuracy: 0.8471 - val_loss: nan - val_accuracy: 0.8244
Epoch 4/5
12/12 [==============================] - 167s 14s/step - loss: nan - accuracy: 0.8471 - val_loss: nan - val_accuracy: 0.8244
Epoch 5/5
12/12 [==============================] - 167s 14s/step - loss: nan - accuracy: 0.8471 - val_loss: nan - val_accuracy: 0.8244

/usr/local/lib/python3.6/dist-packages/kerastuner/engine/metrics_tracking.py:92: RuntimeWarning: All-NaN axis encountered
  return np.nanmin(values)

Trial complete
Trial summary
|-Trial ID: 14bd58fa484de9c1d1133629d5116363
|-Score: 0.8244484066963196
|-Best step: 0
Hyperparameters:
|-activation_conv_index_0_etg_0_desc: selu
|-activation_conv_index_0_etg_0_remont: tanh
|-activation_conv_index_0_etg_1_desc: elu
|-activation_conv_index_0_etg_1_remont: elu
|-activation_conv_index_0_etg_2_desc: elu
|-activation_conv_index_0_etg_2_remont: linear
|-activation_conv_index_0_etg_3_desc: relu
|-activation_conv_index_0_etg_3_remont: tanh
|-activation_conv_index_0_etg_4_desc: selu
|-activation_conv_index_0_etg_4_remont: tanh
|-activation_conv_index_0_etg_5_desc: selu
|-activation_conv_index_0_etg_5_remont: linear
|-activation_conv_index_0_etg_6_desc: elu
|-activation_conv_index_0_etg_6_remont: selu
|-activation_conv_index_0_etg_7_desc: relu
|-activation_conv_index_0_etg_7_remont: relu
|-activation_conv_index_0_etg_8_desc: relu
|-activation_conv_index_1_etg_0_desc: elu
|-activation_conv_index_1_etg_0_remont: relu
|-activation_conv_index_1_etg_1_desc: selu
|-activation_conv_index_1_etg_1_remont: linear
|-activation_conv_index_1_etg_2_desc: relu
|-activation_conv_index_1_etg_2_remont: relu
|-activation_conv_index_1_etg_3_desc: tanh
|-activation_conv_index_1_etg_3_remont: elu
|-activation_conv_index_1_etg_4_desc: elu
|-activation_conv_index_2_etg_0_desc: elu
|-activation_conv_index_2_etg_1_desc: elu
|-activation_conv_index_2_etg_1_remont: linear
|-activation_conv_index_2_etg_2_desc: linear
|-activation_conv_index_2_etg_2_remont: relu
|-activation_conv_index_2_etg_3_desc: selu
|-activation_conv_index_2_etg_3_remont: relu
|-activation_conv_index_2_etg_4_desc: tanh
|-activation_conv_index_3_etg_0_desc: selu
|-activation_conv_index_3_etg_1_remont: linear
|-beta_1: 0.9936631731936973
|-beta_2: 1.1187660572787035
|-filtre_conv_index_0_etg_0_desc: 50
|-filtre_conv_index_0_etg_0_remont: 500
|-filtre_conv_index_0_etg_1_desc: 500
|-filtre_conv_index_0_etg_1_remont: 1
|-filtre_conv_index_0_etg_2_desc: 1
|-filtre_conv_index_0_etg_2_remont: 100
|-filtre_conv_index_0_etg_3_desc: 1
|-filtre_conv_index_0_etg_3_remont: 100
|-filtre_conv_index_0_etg_4_desc: 3
|-filtre_conv_index_0_etg_4_remont: 100
|-filtre_conv_index_0_etg_5_desc: 100
|-filtre_conv_index_0_etg_5_remont: 1
|-filtre_conv_index_0_etg_6_desc: 50
|-filtre_conv_index_0_etg_6_remont: 10
|-filtre_conv_index_0_etg_7_desc: 10
|-filtre_conv_index_0_etg_7_remont: 50
|-filtre_conv_index_0_etg_8_desc: 3
|-filtre_conv_index_1_etg_0_desc: 100
|-filtre_conv_index_1_etg_0_remont: 1
|-filtre_conv_index_1_etg_1_desc: 50
|-filtre_conv_index_1_etg_1_remont: 100
|-filtre_conv_index_1_etg_2_desc: 100
|-filtre_conv_index_1_etg_2_remont: 500
|-filtre_conv_index_1_etg_3_desc: 50
|-filtre_conv_index_1_etg_3_remont: 500
|-filtre_conv_index_1_etg_4_desc: 3
|-filtre_conv_index_2_etg_0_desc: 500
|-filtre_conv_index_2_etg_1_desc: 3
|-filtre_conv_index_2_etg_1_remont: 10
|-filtre_conv_index_2_etg_2_desc: 1
|-filtre_conv_index_2_etg_2_remont: 1
|-filtre_conv_index_2_etg_3_desc: 1
|-filtre_conv_index_2_etg_3_remont: 1
|-filtre_conv_index_2_etg_4_desc: 50
|-filtre_conv_index_3_etg_0_desc: 3
|-filtre_conv_index_3_etg_1_remont: 50
|-filtre_deconv_etg_0_remont: 3
|-filtre_deconv_etg_1_remont: 1
|-filtre_deconv_etg_2_remont: 1
|-filtre_deconv_etg_3_remont: 500
|-filtre_deconv_etg_4_remont: 500
|-filtre_deconv_etg_5_remont: 10
|-filtre_deconv_etg_6_remont: 1
|-filtre_deconv_etg_7_remont: 1
|-kernel_conv_index_0_etg_0_desc: 3
|-kernel_conv_index_0_etg_0_remont: 3
|-kernel_conv_index_0_etg_1_desc: 3
|-kernel_conv_index_0_etg_1_remont: 3
|-kernel_conv_index_0_etg_2_desc: 2
|-kernel_conv_index_0_etg_2_remont: 3
|-kernel_conv_index_0_etg_3_desc: 3
|-kernel_conv_index_0_etg_3_remont: 2
|-kernel_conv_index_0_etg_4_desc: 3
|-kernel_conv_index_0_etg_4_remont: 2
|-kernel_conv_index_0_etg_5_desc: 2
|-kernel_conv_index_0_etg_5_remont: 2
|-kernel_conv_index_0_etg_6_desc: 2
|-kernel_conv_index_0_etg_6_remont: 2
|-kernel_conv_index_0_etg_7_desc: 2
|-kernel_conv_index_0_etg_7_remont: 2
|-kernel_conv_index_0_etg_8_desc: 3
|-kernel_conv_index_1_etg_0_desc: 3
|-kernel_conv_index_1_etg_0_remont: 2
|-kernel_conv_index_1_etg_1_desc: 3
|-kernel_conv_index_1_etg_1_remont: 3
|-kernel_conv_index_1_etg_2_desc: 2
|-kernel_conv_index_1_etg_2_remont: 3
|-kernel_conv_index_1_etg_3_desc: 2
|-kernel_conv_index_1_etg_3_remont: 3
|-kernel_conv_index_1_etg_4_desc: 2
|-kernel_conv_index_2_etg_0_desc: 3
|-kernel_conv_index_2_etg_1_desc: 3
|-kernel_conv_index_2_etg_1_remont: 3
|-kernel_conv_index_2_etg_2_desc: 3
|-kernel_conv_index_2_etg_2_remont: 2
|-kernel_conv_index_2_etg_3_desc: 3
|-kernel_conv_index_2_etg_3_remont: 2
|-kernel_conv_index_2_etg_4_desc: 3
|-kernel_conv_index_3_etg_0_desc: 3
|-kernel_conv_index_3_etg_1_remont: 2
|-learning_rate: 0.0001
|-nb_conv_etg_0_desc: 2
|-nb_conv_etg_0_remont: 2
|-nb_conv_etg_1_desc: 3
|-nb_conv_etg_1_remont: 1
|-nb_conv_etg_2_desc: 1
|-nb_conv_etg_2_remont: 0
|-nb_conv_etg_3_desc: 3
|-nb_conv_etg_3_remont: 0
|-nb_conv_etg_4_desc: 3
|-nb_conv_etg_4_remont: 4
|-nb_conv_etg_5_desc: 3
|-nb_conv_etg_5_remont: 2
|-nb_conv_etg_6_desc: 2
|-nb_conv_etg_6_remont: 1
|-nb_conv_etg_7_desc: 4
|-nb_conv_etg_7_remont: 3
|-nb_conv_etg_8_desc: 3
|-nb_pool: 2
|-pool_type_0: avg
|-pool_type_1: avg
|-pool_type_2: avg
|-pool_type_3: max
|-pool_type_4: max
|-pool_type_5: avg
|-pool_type_6: max
|-pool_type_7: max

Epoch 1/5
12/12 [==============================] - 1895s 158s/step - loss: nan - accuracy: 0.8352 - val_loss: nan - val_accuracy: 0.8244
Epoch 2/5
12/12 [==============================] - 1897s 158s/step - loss: nan - accuracy: 0.8471 - val_loss: nan - val_accuracy: 0.8244
Epoch 3/5
12/12 [==============================] - 1901s 158s/step - loss: nan - accuracy: 0.8471 - val_loss: nan - val_accuracy: 0.8244
Epoch 4/5
12/12 [==============================] - 1889s 157s/step - loss: nan - accuracy: 0.8471 - val_loss: nan - val_accuracy: 0.8244
Epoch 5/5
12/12 [==============================] - 1898s 158s/step - loss: nan - accuracy: 0.8471 - val_loss: nan - val_accuracy: 0.8244
Epoch 1/5
12/12 [==============================] - 1902s 159s/step - loss: nan - accuracy: 0.8312 - val_loss: nan - val_accuracy: 0.8244
Epoch 2/5
12/12 [==============================] - 1893s 158s/step - loss: nan - accuracy: 0.8471 - val_loss: nan - val_accuracy: 0.8244
Epoch 3/5
12/12 [==============================] - 1893s 158s/step - loss: nan - accuracy: 0.8471 - val_loss: nan - val_accuracy: 0.8244
Epoch 4/5
12/12 [==============================] - 1897s 158s/step - loss: nan - accuracy: 0.8471 - val_loss: nan - val_accuracy: 0.8244
Epoch 5/5
 6/12 [==============>...............] - ETA: 10:47 - loss: nan - accuracy: 0.8727

# Reconstruction du modèle le plus performant

In [0]:
#@title meilleur resultat max 3 couche de conv : parser avec a { display-mode: "form" }
a = """Trial summary
|-Trial ID: e3de1ef4d6472fd0557549403c1a956e
|-Score: 0.846445620059967
|-Best step: 0
Hyperparameters:
|-activation_conv_index_0_etg_0_desc: relu
|-activation_conv_index_0_etg_0_remont: relu
|-activation_conv_index_0_etg_1_desc: linear
|-activation_conv_index_0_etg_1_remont: linear
|-activation_conv_index_0_etg_2_desc: selu
|-activation_conv_index_0_etg_2_remont: relu
|-activation_conv_index_0_etg_3_desc: linear
|-activation_conv_index_0_etg_3_remont: relu
|-activation_conv_index_0_etg_4_desc: elu
|-activation_conv_index_0_etg_4_remont: relu
|-activation_conv_index_0_etg_5_desc: relu
|-activation_conv_index_0_etg_5_remont: relu
|-activation_conv_index_0_etg_6_desc: relu
|-activation_conv_index_0_etg_6_remont: relu
|-activation_conv_index_0_etg_7_desc: relu
|-activation_conv_index_0_etg_7_remont: relu
|-activation_conv_index_0_etg_8_desc: relu
|-activation_conv_index_1_etg_0_desc: selu
|-activation_conv_index_1_etg_0_remont: relu
|-activation_conv_index_1_etg_1_desc: linear
|-activation_conv_index_1_etg_1_remont: relu
|-activation_conv_index_1_etg_2_desc: relu
|-activation_conv_index_1_etg_2_remont: elu
|-activation_conv_index_1_etg_3_desc: relu
|-activation_conv_index_1_etg_3_remont: relu
|-activation_conv_index_2_etg_0_desc: selu
|-activation_conv_index_2_etg_0_remont: elu
|-activation_conv_index_2_etg_1_desc: tanh
|-activation_conv_index_2_etg_2_desc: relu
|-activation_conv_index_2_etg_2_remont: tanh
|-activation_conv_index_2_etg_3_desc: relu
|-activation_conv_index_2_etg_3_remont: relu
|-activation_conv_index_3_etg_0_remont: selu
|-activation_conv_index_3_etg_2_desc: relu
|-beta_1: 1.1567454740840701
|-beta_2: 1.1580270355932856
|-filtre_conv_index_0_etg_0_desc: 100
|-filtre_conv_index_0_etg_0_remont: 100
|-filtre_conv_index_0_etg_1_desc: 1
|-filtre_conv_index_0_etg_1_remont: 100
|-filtre_conv_index_0_etg_2_desc: 10
|-filtre_conv_index_0_etg_2_remont: 3
|-filtre_conv_index_0_etg_3_desc: 3
|-filtre_conv_index_0_etg_3_remont: 50
|-filtre_conv_index_0_etg_4_desc: 500
|-filtre_conv_index_0_etg_4_remont: 1
|-filtre_conv_index_0_etg_5_desc: 1
|-filtre_conv_index_0_etg_5_remont: 1
|-filtre_conv_index_0_etg_6_desc: 1
|-filtre_conv_index_0_etg_6_remont: 1
|-filtre_conv_index_0_etg_7_desc: 1
|-filtre_conv_index_0_etg_7_remont: 1
|-filtre_conv_index_0_etg_8_desc: 1
|-filtre_conv_index_1_etg_0_desc: 50
|-filtre_conv_index_1_etg_0_remont: 50
|-filtre_conv_index_1_etg_1_desc: 500
|-filtre_conv_index_1_etg_1_remont: 1
|-filtre_conv_index_1_etg_2_desc: 1
|-filtre_conv_index_1_etg_2_remont: 50
|-filtre_conv_index_1_etg_3_desc: 1
|-filtre_conv_index_1_etg_3_remont: 1
|-filtre_conv_index_2_etg_0_desc: 500
|-filtre_conv_index_2_etg_0_remont: 50
|-filtre_conv_index_2_etg_1_desc: 10
|-filtre_conv_index_2_etg_2_desc: 1
|-filtre_conv_index_2_etg_2_remont: 100
|-filtre_conv_index_2_etg_3_desc: 1
|-filtre_conv_index_2_etg_3_remont: 1
|-filtre_conv_index_3_etg_0_remont: 3
|-filtre_conv_index_3_etg_2_desc: 1
|-filtre_deconv_etg_0_remont: 1
|-filtre_deconv_etg_1_remont: 3
|-filtre_deconv_etg_2_remont: 50
|-filtre_deconv_etg_3_remont: 50
|-filtre_deconv_etg_4_remont: 1
|-filtre_deconv_etg_5_remont: 1
|-filtre_deconv_etg_6_remont: 1
|-filtre_deconv_etg_7_remont: 1
|-kernel_conv_index_0_etg_0_desc: 3
|-kernel_conv_index_0_etg_0_remont: 2
|-kernel_conv_index_0_etg_1_desc: 3
|-kernel_conv_index_0_etg_1_remont: 3
|-kernel_conv_index_0_etg_2_desc: 3
|-kernel_conv_index_0_etg_2_remont: 2
|-kernel_conv_index_0_etg_3_desc: 2
|-kernel_conv_index_0_etg_3_remont: 2
|-kernel_conv_index_0_etg_4_desc: 3
|-kernel_conv_index_0_etg_4_remont: 2
|-kernel_conv_index_0_etg_5_desc: 2
|-kernel_conv_index_0_etg_5_remont: 2
|-kernel_conv_index_0_etg_6_desc: 2
|-kernel_conv_index_0_etg_6_remont: 2
|-kernel_conv_index_0_etg_7_desc: 2
|-kernel_conv_index_0_etg_7_remont: 2
|-kernel_conv_index_0_etg_8_desc: 2
|-kernel_conv_index_1_etg_0_desc: 3
|-kernel_conv_index_1_etg_0_remont: 3
|-kernel_conv_index_1_etg_1_desc: 2
|-kernel_conv_index_1_etg_1_remont: 2
|-kernel_conv_index_1_etg_2_desc: 2
|-kernel_conv_index_1_etg_2_remont: 2
|-kernel_conv_index_1_etg_3_desc: 2
|-kernel_conv_index_1_etg_3_remont: 2
|-kernel_conv_index_2_etg_0_desc: 3
|-kernel_conv_index_2_etg_0_remont: 3
|-kernel_conv_index_2_etg_1_desc: 2
|-kernel_conv_index_2_etg_2_desc: 2
|-kernel_conv_index_2_etg_2_remont: 3
|-kernel_conv_index_2_etg_3_desc: 2
|-kernel_conv_index_2_etg_3_remont: 2
|-kernel_conv_index_3_etg_0_remont: 2
|-kernel_conv_index_3_etg_2_desc: 2
|-learning_rate: 1e-05
|-nb_conv_etg_0_desc: 3
|-nb_conv_etg_0_remont: 4
|-nb_conv_etg_1_desc: 2
|-nb_conv_etg_1_remont: 2
|-nb_conv_etg_2_desc: 4
|-nb_conv_etg_2_remont: 2
|-nb_conv_etg_3_desc: 3
|-nb_conv_etg_3_remont: 3
|-nb_conv_etg_4_desc: 0
|-nb_conv_etg_4_remont: 1
|-nb_conv_etg_5_desc: 1
|-nb_conv_etg_5_remont: 1
|-nb_conv_etg_6_desc: 1
|-nb_conv_etg_6_remont: 1
|-nb_conv_etg_7_desc: 1
|-nb_conv_etg_7_remont: 1
|-nb_conv_etg_8_desc: 1
|-nb_pool: 8
|-pool_type_0: max
|-pool_type_1: max
|-pool_type_2: max
|-pool_type_3: max
|-pool_type_4: max
|-pool_type_5: max
|-pool_type_6: max
|-pool_type_7: max"""
a = a.split("\n")


In [15]:
for l in a:
    if "etg_0" in l and "desc" in l:
        print(l)

|-activation_conv_index_0_etg_0_desc: relu
|-activation_conv_index_1_etg_0_desc: selu
|-activation_conv_index_2_etg_0_desc: selu
|-filtre_conv_index_0_etg_0_desc: 100
|-filtre_conv_index_1_etg_0_desc: 50
|-filtre_conv_index_2_etg_0_desc: 500
|-kernel_conv_index_0_etg_0_desc: 3
|-kernel_conv_index_1_etg_0_desc: 3
|-kernel_conv_index_2_etg_0_desc: 3
|-nb_conv_etg_0_desc: 3
